#**PHASE D'AMORCE**

## **INITIALISATION**

In [ ]:
import sys # <--- Importer le module sys
new_limit = 10 * 1024 * 1024


In [2]:
import csv
import os
import sys # <--- Importer le module sys

# --- Configuration ---
input_filename = '/app/DATA/hard_right_wing_all_transcripts.csv'
output_filename = '/app/DATA/filtered_transcripts.csv'
column_to_check = 'channel_title'
value_to_remove = 'Jordan B Peterson'
delimiter = ';'

# --- Augmenter la limite de taille de champ CSV ---
# Essayez d'augmenter la limite. Commencez par une valeur plus grande,
# par exemple 1 Mo (1024*1024 bytes) ou 10 Mo (10*1024*1024).
# Si l'erreur persiste, augmentez davantage cette valeur.
# Attention : une valeur trop grande peut consommer beaucoup de mémoire.
# Vous pouvez aussi essayer la limite maximale théorique : sys.maxsize
# mais commencez par une valeur définie plus raisonnable.

# Tentative avec 10 Mo (ajustez si nécessaire) :
new_limit = 10 * 1024 * 1024
try:
    # Boucle pour gérer les cas où même sys.maxsize n'est pas suffisant (très rare)
    # ou pour trouver une limite fonctionnelle de manière itérative si on ne met pas sys.maxsize directement
    current_limit = csv.field_size_limit()
    while new_limit > current_limit:
        try:
            csv.field_size_limit(new_limit)
            print(f"Limite de taille de champ CSV augmentée à {new_limit} bytes")
            break # Sortir si le réglage a fonctionné
        except OverflowError:
            # Si new_limit est trop grand pour le système (ex: sys.maxsize sur certains systèmes 32 bits)
            # Réduire la limite par étapes peut être une stratégie, mais souvent on met sys.maxsize
            print(f"Impossible de définir la limite à {new_limit}, tentative avec une valeur légèrement inférieure ou sys.maxsize.")
            # Pour simplifier ici, on peut directement essayer sys.maxsize
            # ou simplement réduire la valeur manuellement si on sait qu'elle est trop grande.
            # Mettons sys.maxsize comme alternative courante:
            try:
                csv.field_size_limit(sys.maxsize)
                print(f"Limite de taille de champ CSV augmentée à sys.maxsize ({sys.maxsize} bytes)")
                new_limit = sys.maxsize # Mettre à jour new_limit pour sortir de la boucle
            except OverflowError:
                 print("ERREUR CRITIQUE : Impossible d'augmenter suffisamment la limite de taille de champ CSV.")
                 raise # Relancer l'erreur si même sys.maxsize échoue
            break # Sortir après avoir tenté sys.maxsize

# Gestion plus simple (souvent suffisante):
# try:
#    csv.field_size_limit(new_limit) # Essayez avec 10MB
# except OverflowError:
#    csv.field_size_limit(sys.maxsize) # Si 10MB trop grand (peu probable), essayez max

except Exception as e:
    print(f"Erreur lors de la modification de la limite de taille de champ : {e}")
    # Optionnel: quitter si on ne peut pas changer la limite
    # raise SystemExit()


# --- Vérification de l'existence du fichier d'entrée ---
if not os.path.exists(input_filename):
    print(f"Erreur : Le fichier d'entrée '{input_filename}' n'a pas été trouvé.")
else:
    try:
        # --- Traitement ---
        rows_written = 0
        rows_read = 0
        rows_removed = 0

        # Ouvre le fichier d'entrée en lecture et le fichier de sortie en écriture
        with open(input_filename, mode='r', newline='', encoding='utf-8') as infile, \
             open(output_filename, mode='w', newline='', encoding='utf-8') as outfile:

            # Crée un lecteur CSV pour lire le fichier d'entrée
            # NOTE: Le lecteur utilisera la nouvelle limite de champ définie globalement
            reader = csv.reader(infile, delimiter=delimiter)

            # Crée un écrivain CSV pour écrire dans le fichier de sortie
            writer = csv.writer(outfile, delimiter=delimiter)

            # Lit la ligne d'en-tête
            header = next(reader)
            rows_read += 1

            # Trouve l'index de la colonne 'channel_title'
            try:
                channel_title_index = header.index(column_to_check)
            except ValueError:
                print(f"Erreur : La colonne '{column_to_check}' n'existe pas dans l'en-tête du fichier.")
                exit()

            # Écrit l'en-tête dans le fichier de sortie
            writer.writerow(header)
            rows_written += 1

            # Parcourt chaque ligne restante dans le fichier d'entrée
            for row in reader:
                rows_read += 1
                if len(row) > channel_title_index:
                    if row[channel_title_index] != value_to_remove:
                        writer.writerow(row)
                        rows_written += 1
                    else:
                        rows_removed += 1
                else:
                    print(f"Avertissement : Ligne {rows_read} ignorée car elle a moins de colonnes que prévu.")

        # --- Fin du traitement ---
        print("-" * 30)
        print("Traitement terminé.")
        print(f"Fichier d'entrée : '{input_filename}'")
        print(f"Fichier de sortie : '{output_filename}'")
        print(f"Lignes lues au total : {rows_read}")
        print(f"Lignes contenant '{value_to_remove}' supprimées : {rows_removed}")
        print(f"Lignes écrites dans le nouveau fichier : {rows_written}")
        print("-" * 30)

    except FileNotFoundError:
        print(f"Erreur : Le fichier d'entrée '{input_filename}' n'a pas été trouvé lors de l'ouverture.")
    except csv.Error as e: # Capture spécifique de l'erreur CSV
        print(f"Erreur CSV lors de la lecture de la ligne {rows_read+1}: {e}")
        print("Cela peut se produire si la limite de champ est encore trop petite ou si le fichier CSV est mal formé.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue (ligne approx {rows_read+1}): {e}")

Limite de taille de champ CSV augmentée à 10485760 bytes
------------------------------
Traitement terminé.
Fichier d'entrée : '/app/DATA/hard_right_wing_all_transcripts.csv'
Fichier de sortie : '/app/DATA/filtered_transcripts.csv'
Lignes lues au total : 35730
Lignes contenant 'Jordan B Peterson' supprimées : 1204
Lignes écrites dans le nouveau fichier : 34526
------------------------------


In [2]:
!pip install -U ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.9 MB/s eta 0:00:00a 0:00:01


In [1]:
# --- Bibliothèque standard ---
from collections import Counter, defaultdict
import contextlib
import copy
import csv
from datetime import datetime, timedelta
from functools import partial
import gc
import html
import importlib
import io
import itertools
import json
import locale
import logging
import math
import multiprocessing
from multiprocessing import cpu_count
import os
import pickle
import random
import re
import string
import subprocess
import time
from urllib import request # UTILE ?
import spacy
from smart_open import open
import openai
from openai import OpenAIError
import time
import numpy as np
from collections import deque
from sklearn.metrics.pairwise import cosine_similarity
import tiktoken
import matplotlib.pyplot as plt
import math
from collections import Counter
import contextlib

# --- Bibliothèques externes ---
from bs4 import BeautifulSoup
from charset_normalizer import from_path
from datasketch import MinHash, MinHashLSH
from dateutil import parser
from dateutil.parser import parse
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.gridspec import GridSpec
import numpy as np
from ortools.linear_solver import pywraplp
import pandas as pd
import psutil
from pyate import cvalues
from pyate.term_extraction_pipeline import TermExtractionPipeline
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.ndimage import gaussian_filter
from scipy.spatial import distance
from scipy.spatial.distance import cosine, pdist
import seaborn as sns
from spacy.language import Language
from spacy.tokens import Doc
from sklearn.decomposition import NMF, PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
    TfidfVectorizer
)
from sklearn.inspection import permutation_importance
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import (
    cosine_distances,
    cosine_similarity,
    manhattan_distances
)
from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_score
)
from sklearn.preprocessing import label_binarize
from statsmodels.stats.outliers_influence import variance_inflation_factor
import torch
#import torchaudio
#import torchvision
from tqdm.auto import tqdm, trange
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline
)
import unidecode
from octis.evaluation_metrics.coherence_metrics import Coherence

import time
import numpy as np
from collections import deque
from sklearn.metrics.pairwise import cosine_similarity
import tiktoken

In [2]:
folder_path = '/app/'

In [3]:
PX_PER_TOPIC = 60      # Hauteur en pixels par topic
DPI = 72*4              # Résolution : 300 dpi
FIGURE_WIDTH_INCH = 6.30 # A4 AVEC 2.5CM DE MARGES #4.02 UN GALLIMARD AVEC 2.4CM DE MARGES # Largeur fixe (en pouces) que vous souhaitez

In [4]:
# Actuellement, seuls les langues suivantes sont prises en charge :
# français, anglais, espagnol, allemand, catalan, chinois, danois, japonais, slovaque, et ukrainien.
# Codes linguistiques correspondants : fr, en, es, de, ca, zh, da, ja, sl, uk.
language = 'en'

In [5]:
import matplotlib
from matplotlib import font_manager, rcParams

matplotlib.rcParams['font.size'] = 10.5

# Spécifiez le chemin vers le fichier de police
font_path = folder_path + "Roboto/XCharter-Roman.otf"
font_manager.fontManager.addfont(font_path)
xcharter_font = font_manager.FontProperties(fname=font_path)
matplotlib.rcParams['font.family'] = xcharter_font.get_name()

In [6]:
!python -m spacy download en_core_web_sm
nlp_pipeline = spacy.load("en_core_web_sm", exclude=["ner"])

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [23]:
#nlp_pipeline = None
file_to_run = f"{folder_path}lexical_topic_modeling_backend.ipynb"
%run "$file_to_run"

## **DÉFINITION DES PARAMÈTRES**

In [8]:
# **grammatical_classes** : list[str], optionnel
#     Une liste de classes grammaticales à conserver lors de la construction des vecteurs TF-IDF.
#     Les classes doivent correspondre aux catégories grammaticales reconnues par spaCy.
#     Par défaut, les classes `['NOUN', 'PROPN', 'VERB']` sont utilisées, ciblant les noms communs,
#     noms propres et verbes.
#
# ### Classes grammaticales reconnues par spaCy :
# ------------------------------------------
# - 'NOUN' : Noms communs
# - 'PROPN' : Noms propres
# - 'VERB' : Verbes
# - 'ADJ' : Adjectifs
# - 'ADV' : Adverbes
# - 'PRON' : Pronoms
# - 'DET' : Déterminants (articles)
# - 'ADP' : Prépositions ou postpositions
# - 'CONJ' : Conjonctions de coordination
# - 'CCONJ' : Conjonctions de coordination (UD)
# - 'SCONJ' : Conjonctions de subordination (UD)
# - 'AUX' : Verbes auxiliaires (ex. "être", "avoir")
# - 'PART' : Particules
# - 'INTJ' : Interjections
# - 'NUM' : Nombres
# - 'SYM' : Symboles
# - 'X' : Autres (inclassables)
#
# ### Retourne :
# ---------
# **tuple** :
#     Un tuple contenant les résultats de la vectorisation TF-IDF :
#     - **tfidf_vectorizer** : L'objet `TfidfVectorizer` utilisé pour la vectorisation.
#     - **tfidf** : Une matrice sparse TF-IDF des documents vectorisés.
#     - **tfidf_feature_names** : La liste des unigrams sélectionnés comme caractéristiques.
#     - **tokenized_documents** : Une version tokenisée et filtrée des documents d'entrée, limitée
#       aux classes grammaticales spécifiées.
#
# ### Exemple :
# ---------
# Pour effectuer une vectorisation TF-IDF sur les catégories grammaticales 'NOUN' et 'PROPN' :
# grammatical_classes = ['NOUN', 'PROPN']
grammatical_classes = ['NOUN', 'PROPN', 'VERB']

In [9]:
# La variable `threshold` fixe le seuil minimal d'articles requis pour qu'un journal
# soit inclus dans certaines analyses spécifiques :
# - Les journaux ayant un nombre d'articles inférieur à ce seuil ne seront pas pris
#   en compte dans les visualisations portant sur les journaux (exemple : graphiques
#   comparant des volumes de publications ou des analyses statistiques spécifiques).
# - Cela n'exclut pas ces journaux du corpus pour d'autres types d'analyse, comme le topic modeling,
#   où tous les articles restent intégrés indépendamment de leur origine.
#
# Ce seuil permet de concentrer les analyses et tests statistiques sur des journaux
# suffisamment représentatifs, en évitant que des titres marginalement présents
# introduisent du bruit dans les résultats.
threshold = 10

In [10]:
# Ces paramètres s'appliquent principalement aux corpus issus d'Europresse.
# Par exemple, les articles contenant moins de 500 caractères ou plus de 100 000 caractères
# sont considérés comme suspects et sont donc supprimés du corpus.
#
# Ces valeurs doivent être ajustées en fonction du type de corpus analysé,
# car elles peuvent varier significativement selon les sources et les objectifs de l'étude.
#
# Pour les corpus provenant d'Europresse, l'algorithme élimine systématiquement les articles
# dont le nombre de caractères est inférieur à `minimum_caracters_nb_by_document` ou supérieur à
# `maximum_caracters_nb_by_document`. Cette approche repose sur trois considérations :
#
# 1. **Suspicion de contenu non standard** : Un article comportant moins de 500 caractères ou
#    plus de 100 000 caractères s'écarte des normes usuelles. Ces cas extrêmes suggèrent des anomalies
#    (ex. : méta-données mal extraites, textes hors contexte ou dégradés).
#
# 2. **Analyse thématique limitée** : Un article très court (moins de 100 ou 200 caractères,
#    soit environ vingt mots) ne fournit pas suffisamment de matière pour une analyse lexicale
#    ou thématique pertinente. Cela revient à une situation comparable au rejet d'une analyse
#    du chi² pour des croisements avec des effectifs insuffisants.
#
# 3. **Limitation technique de l’algorithme de NER (Reconnaissance d’Entités Nommées)** :
#    Les outils de NER utilisés (par exemple, spaCy, Stanford NER, etc.) peuvent présenter
#    des contraintes de longueur. Au-delà de 100 000 caractères, l’algorithme risque de ne plus
#    pouvoir traiter correctement le document, entraînant un blocage technique. Le seuil
#    maximum de 100 000 caractères vise donc également à préserver la faisabilité et la
#    qualité de l’analyse NER.
minimum_caracters_nb_by_document = 2000
maximum_caracters_nb_by_document = 100000000

In [11]:
# Ce paramètre détermine si les doublons doivent être conservés ou non.


# - À False : les doublons sont conservés.
# - À True : les doublons sont supprimés.
#
# La valeur True est particulièrement pertinente pour les corpus issus d'Europresse.
# Elle permet de gérer plusieurs problèmes liés à la collecte des données :
# - Élimination des doublons potentiels causés par le fonctionnement spécifique d'Europresse.
# - Suppression des recouvrements partiels entre différents fichiers d'articles,
#   évitant ainsi les répétitions accidentelles.
# - Identification et retrait des articles produits par simple copié/collé,
#   grâce à un algorithme qui résiste aux variations mineures. Ainsi, les articles
#   n'apportant aucune information substantiellement nouvelle sont exclus du corpus.
#
# Toutefois, conserver les doublons (valeur False) peut également être justifié,
# car ces répétitions reflètent une certaine dynamique médiatique. Dans ce cas,
# le paramètre devra rester à False pour intégrer ces éléments au corpus.
go_remove_duplicates = False

In [12]:
# La variable `web_paper_differentiation` détermine si les versions papier et web

# d'une même revue doivent être fusionnées ou conservées distinctes :
# - `False` : les versions papier et web sont fusionnées en un seul ensemble.
# - `True` : les versions papier et web sont traitées comme des entités distinctes.
#
# Ce choix dépend de la finalité de l'analyse :
# - Si l'objectif est de considérer un média dans son ensemble, sans distinction
#   entre ses formats, laissez cette variable à `False`.
# - Si l'on souhaite analyser les spécificités des contenus selon leur support
#   (papier ou web), définissez cette variable à `True`.
web_paper_differentiation = False

In [13]:
# La variable `source_type` indique l'origine du corpus et ajuste le traitement des données en conséquence.
#
# Valeurs possibles pour `source_type` :
# - "europresse" : le corpus provient de la plateforme Europresse.
#   - Des prétraitements spécifiques sont appliqués pour gérer les particularités de cette source,
#     telles que le format des fichiers ou les métadonnées propres à Europresse.
# - "istex" : le corpus provient de la plateforme ISTEX.
#   - Des étapes adaptées à la structure des métadonnées et des documents fournis par ISTEX sont appliquées.
# - "csv" : le corpus provient d'un fichier CSV ou d'autres formats standards.
#   - Le séparateur du fichier doit être une virgule ou un point-virgule. Le fichier doit également
#     contenir a minima deux colonnes : une colonne "text" et une colonne "date".
#
# Le choix de `source_type` influence directement les étapes de prétraitement et d'analyse des données.
source_type = 'csv'

In [14]:
# La variable `base_name` correspond au nom du corpus. Elle doit être modifiée de manière à

# décrire précisément le corpus analysé, en suivant des bonnes pratiques de nommage.
# Exemple : "ukraine_russie__presse_francilienne__fev2022_feb2023".
#
# Le contenu de `base_name` doit être présent dans les noms des fichiers situés dans le dossier "DATA"
# et utilisés pour l'analyse. L'algorithme est conçu pour gérer des corpus répartis sur plusieurs fichiers :
# il suffit que tous les fichiers concernés contiennent le texte exact de `base_name` dans leur nom.
#
# Exemple de fichiers conformes :
# - "ukraine_russie__presse_francilienne__fev2022_mars2024__feb2022_sep2022.HTML"
# - "ukraine_russie__presse_francilienne__fev2022_feb2023__oct2022_feb2023.HTML"
#
# Ces fichiers seront inclus dans l'analyse dès lors qu'ils partagent le même `base_name`.
#
# De plus, les résultats de l'analyse seront stockés dans un dossier spécifique dont le chemin
# est défini par la variable `RESULTS_PATH`. Ce chemin inclut le `base_name` pour assurer une
# organisation cohérente des résultats.
base_name = 'hard_right_wing_all'
results_path = folder_path + "RESULTS_" + base_name + "/"

In [15]:
# Cette fonction crée un dossier de résultats basé sur le nom spécifié dans `base_name`.

# Elle vérifie si le dossier existe déjà, et si ce n'est pas le cas, elle le crée.
# Le nom du dossier sera préfixé par "RESULTS_" et inclura le contenu de `base_name`.
# Elle définit également le nom du fichier CSV où les résultats seront sauvegardés.
create_results_folder(base_name)# CA PEUT PASSER DANS LA LECTURE DES DOCUMENTS

## **PRÉPARATION DES DOCUMENTS**

In [16]:
# Initialisation des variables pour stocker les documents, les objets BeautifulSoup associés,
# et les métadonnées extraites des différentes sources de données (europresse, CSV, ISTEX).
#
# - 'documents' : liste qui contiendra les textes extraits des documents.
#    - Pour 'europresse', ce seront les textes nettoyés extraits des articles HTML.
#    - Pour 'csv', ce seront les contenus des colonnes 'text' ou 'description' des fichiers CSV filtrés par le nombre de caractères.
#    - Pour 'istex', ce seront les contenus texte extraits des fichiers `.txt` associés aux fichiers `.json` correspondants.
documents = []

# - 'all_soups' : liste qui contiendra les objets BeautifulSoup pour chaque document.
#    Cela est utilisé pour manipuler et analyser la structure HTML des documents extraits, en particulier pour 'europresse',
#    où chaque document est transformé en un objet BeautifulSoup afin de procéder à des nettoyages supplémentaires (par exemple, suppression de certains éléments HTML).
all_soups = []

# - 'columns_dict' : dictionnaire qui stocke les métadonnées liées à chaque document.
#    - Pour 'csv', cela contiendra les autres colonnes du fichier (autres que 'text' et 'description') que l'on souhaite conserver comme métadonnées.
#    - Pour 'istex', ce dictionnaire contiendra des champs supplémentaires extraits des fichiers JSON associés à chaque document, tels que 'doi', 'journal', 'date', etc.
columns_dict = {}

# La fonction 'meta_load_documents()' est appelée pour charger et traiter les données des documents à partir de la source spécifiée
# (définie par 'source_type'). En fonction de cette source, les données sont extraites et stockées dans les structures ci-dessus pour un traitement ultérieur.
meta_load_documents()

DOCUMENTS PROCESSÉS:   0%|          | 0/1 [00:00<?, ?it/s]

[*] Taille du fichier CSV = 693.73 Mo
[*] Le fichier est > 200 Mo : lecture directe en UTF-8 (séparateur ';')


27009 documents


#**PHASE DE PRODUCTION DES MATRICES H ET W**

##**LEMMATISATION ET VECTORISATION**

In [17]:
import multiprocessing
import sys

if __name__ == "__main__":
    if sys.platform.startswith('linux') or sys.platform.startswith('darwin'): # Fonctionne sous Linux/macOS
         try:
             multiprocessing.set_start_method('forkserver', force=True)
             print("Méthode de démarrage multiprocessing définie sur 'forkserver'")
         except RuntimeError:
             print("Impossible de redéfinir la méthode de démarrage multiprocessing (déjà démarrée?).")
    # ... reste du code ...

Méthode de démarrage multiprocessing définie sur 'forkserver'


In [18]:
# La fonction process_documents traite une liste de documents en effectuant les opérations suivantes :


# 1. Chargement d'un modèle linguistique spaCy en fonction de la langue spécifiée.
# 2. Pour chaque document, elle effectue :
#    - La lemmatisation des tokens (avec normalisation des noms propres via unidecode).
#    - L'extraction des étiquettes de parties du discours (POS).
#    - La création de listes de mots lemmatisés et normalisés pour chaque phrase.
# 3. Les résultats sont stockés dans quatre listes :
#    - 'documents_lemmatized' (pour les n-grams),
#    - 'all_tab_pos' (mots et leurs POS par document),
#    - 'sentences_norms' (phrases normalisées),
#    - 'all_sentence_pos' (mots et leurs POS par phrase).
# 4. Un suivi de la progression est effectué via tqdm, et des erreurs sont loggées sans interrompre l'exécution.
#documents_lemmatized = []
all_tab_pos = []
sentences_norms = []
all_sentence_pos = []
#process_documents(documents)

##**TF-IDF**

In [19]:
import tempfile

In [24]:
tfidf, tfidf_feature_names, X_sentences, gensim_dictionary, tokenized_corpus, sentence_file_path, sentence_offsets = main_workflow_single_pass(documents, grammatical_classes)

--- Starting Main Workflow (Single SpaCy Pass) ---
Attempting to set spaCy GPU preference...
-> Could not set/check spaCy GPU preference: Cannot use GPU, CuPy is not installed. Relying on pipeline's loaded device.
Using provided spaCy pipeline potentially configured for device: CPU (default)
Temporary sentence file created at: /tmp/sentences_vhu659ht.txt

--- Running Single SpaCy Processing and Data Collection ---
SpaCy pipeline operating on CPU. Using 8 processes for nlp.pipe.
Starting single spaCy processing pass...


Processing Docs (spaCy):   0%|          | 0/27009 [00:00<?, ?it/s]

SpaCy processing complete. Collected data for 3155973 sentences.
Successfully processed 3155973 sentences.
Found 27009 documents in the processed data.

--- Creating Gensim Dictionary and Tokenized Corpus (Document Level) ---
Tokenized corpus created with 27009 documents.
Total tokens in tokenized corpus: 35614172
Gensim Dictionary created with 144341 unique tokens.

--- Vectorizing Documents (TF-IDF) ---
Starting TF-IDF vectorization (level: document)...
Applying CountVectorizer (fit_transform) for level 'document'...


Vectorizing documents:   0%|          | 0/27009 [00:00<?, ?it/s]

Counts matrix shape for level 'document': (27009, 83367)
Applying TfidfTransformer (fit_transform) for level 'document'...
TF-IDF matrix shape for level 'document': (27009, 83367)
Vocabulary size for level 'document': 83367

--- Vectorizing Sentences (TF-IDF) ---
Starting TF-IDF vectorization (level: sentence)...
Applying CountVectorizer (fit_transform) for level 'sentence'...


Vectorizing sentences:   0%|          | 0/3155973 [00:00<?, ?it/s]

Counts matrix shape for level 'sentence': (3155973, 86000)
Applying TfidfTransformer (fit_transform) for level 'sentence'...
TF-IDF matrix shape for level 'sentence': (3155973, 86000)
Vocabulary size for level 'sentence': 86000
Info: Scikit-learn sentence vocabulary (86000 terms) and Gensim dictionary (144341 tokens) may differ due to vectorizer settings (e.g., min_df) or Gensim filtering (if applied).

--- Main Workflow Completed Successfully ---


In [25]:
# --- Cleanup ---
print(f"\nCleaning up temporary sentence file: {sentence_file_path}")
try:
    os.remove(sentence_file_path)
    print("Temporary file removed.")
except OSError as e:
    print(f"Error removing temporary file {sentence_file_path}: {e}")


Cleaning up temporary sentence file: /tmp/sentences_188n59ce.txt
Temporary file removed.


In [ ]:
109minutes

In [ ]:
extract_relevant_sentences_and_titles(
    nmf_models,              # Dictionnaire des modèles NMF entraînés {num_topic: model}
    X_sentences,             # La matrice TF-IDF des phrases (calculée avant)
    sentence_file_path,      # Chemin vers le fichier temporaire des phrases
    sentence_offsets,        # Liste des offsets des phrases dans le fichier
    language,                # Info langue ('fr', 'en', etc.)
    preprompt)

In [ ]:
# Effectue une vectorisation TF-IDF des documents, en se concentrant sur des unigrams correspondant
# aux classes grammaticales spécifiées.
#
# Cette fonction applique une sélection basée sur les classes grammaticales définies par `grammatical_classes`
# afin de limiter la construction des vecteurs TF-IDF aux unigrams pertinents. Cela permet de réduire
# le bruit en excluant les mots non pertinents (par exemple, les déterminants ou les particules)
# et d'améliorer la qualité des résultats analytiques.
unigrams = {}
tfidf, tfidf_feature_names = go_tfidf_vectorization(grammatical_classes)

Mise à jour des unigrams:   0%|          | 0/3 [00:00<?, ?it/s]

Filtrage des stopwords:   0%|          | 0/86338 [00:00<?, ?it/s]

In [30]:
# --- Add this import at the top of your script ---
from sklearn.utils import check_random_state

# --- Fonction d'Initialisation H (Décroissance Items + Bruit) - CORRECTED ---
def create_decreasing_H(n_components, n_features, n_items_decay, decay_power, noise_level, random_state, epsilon, initial_peak):
    """Crée H où chaque ligne suit une tendance décroissante sur les items."""
    rng = check_random_state(random_state)
    base_decay_pattern = np.full(n_features, epsilon)

    # Calculate potential float value
    _actual_decay_items_float = min(n_items_decay, n_features)

    # --- FIX: Cast to integer before use ---
    actual_decay_items = int(_actual_decay_items_float)
    # Note: This truncates. If you prefer rounding, use int(round(_actual_decay_items_float))
    # but simple truncation is usually fine here.

    if actual_decay_items > 1:
        # Now uses the integer version
        item_indices = np.arange(actual_decay_items)
        # Use the float version for potentially more precise calculation if needed,
        # but ensure denominator isn't zero if actual_decay_items_float is exactly 1.0
        denominator = max(_actual_decay_items_float - 1, epsilon) # Avoid division by zero
        decay_values = (1 - item_indices / denominator)**decay_power
        decay_values = (decay_values * (1 - epsilon)) + epsilon
        # Now uses the integer version for slicing
        base_decay_pattern[:actual_decay_items] = decay_values

    # Handle case where int(float_value < 1) becomes 0
    elif actual_decay_items == 1 and n_features > 0:
        base_decay_pattern[0] = initial_peak # Assign high value to the first feature if decay length >= 1
    # If actual_decay_items is 0 (e.g., if n_items_decay was < 1), the pattern remains epsilon

    H = np.zeros((n_components, n_features))
    for k in range(n_components):
        noise = rng.rand(n_features) * noise_level
        H[k, :] = base_decay_pattern + noise
    H = np.maximum(H, epsilon) # Ensure non-negativity
    return H

# --- Fonction d'Initialisation W (Décroissance Topics + Bruit) - Unchanged ---
# (Keep the create_decreasing_W function as you provided it)
def create_decreasing_W(n_samples, n_components, decay_power, noise_level, random_state, epsilon, initial_peak):
    """Crée W où chaque ligne suit une tendance décroissante sur les topics."""
    rng = check_random_state(random_state)
    base_decay_pattern = np.full(n_components, epsilon)
    if n_components > 1:
        topic_indices = np.arange(n_components)
        decay_values = (1 - topic_indices / (n_components - 1))**decay_power
        decay_values = (decay_values * (1 - epsilon)) + epsilon
        base_decay_pattern = decay_values
    elif n_components == 1:
        base_decay_pattern[0] = initial_peak
    W = np.zeros((n_samples, n_components))
    for i in range(n_samples):
        noise = rng.rand(n_components) * noise_level
        W[i, :] = base_decay_pattern + noise
    W = np.maximum(W, epsilon)
    return W

# --- Fonction pour sauvegarder les topics formatés ---
# (Doit être définie avant d'être appelée)
def save_topics_to_file(H_matrix, feature_names, filepath, n_top_words=20):
    """
    Formate et sauvegarde les topics (mots et scores triés) dans un fichier CSV.
    """
    all_topics_dfs = []
    num_topics = H_matrix.shape[0]

    # Assurer que feature_names est une liste ou un array indexable
    if isinstance(feature_names, pd.Series):
        feature_names = feature_names.tolist()
    elif isinstance(feature_names, np.ndarray):
         # Pas besoin de conversion si c'est déjà un ndarray
         pass
    elif not isinstance(feature_names, list):
        print(f"    ATTENTION: feature_names est de type {type(feature_names)}, tentative de conversion en liste.")
        try:
            feature_names = list(feature_names)
        except TypeError:
            print(f"    ERREUR: Impossible de convertir feature_names en liste. Sauvegarde des topics annulée pour {filepath}")
            return # Arrêter la fonction si les noms ne sont pas accessibles

    num_features = len(feature_names)
    if H_matrix.shape[1] != num_features:
         print(f"    ERREUR: Incohérence de dimensions! H_matrix a {H_matrix.shape[1]} features, mais feature_names en a {num_features}. Sauvegarde annulée pour {filepath}")
         return

    n_top_words = min(n_top_words, num_features)

    for topic_idx in range(num_topics):
        topic_vector = H_matrix[topic_idx, :]
        top_indices = topic_vector.argsort()[::-1][:n_top_words]
        try:
            # Tenter de récupérer les mots - peut échouer si les indices sont hors limites
            # (ne devrait pas arriver si les dimensions sont vérifiées, mais sécurité)
            words = [feature_names[i] for i in top_indices]
        except IndexError:
             print(f"    ERREUR: Problème d'index lors de la récupération des mots pour Topic {topic_idx+1}. Vérifiez feature_names.")
             # Mettre des placeholders ou arrêter ? Mettons des placeholders.
             words = [f"ERREUR_INDEX_{i}" for i in top_indices]
        except Exception as e:
             print(f"    ERREUR inattendue lors de la récupération des mots pour Topic {topic_idx+1}: {e}")
             words = [f"ERREUR_MOT_{i}" for i in top_indices]

        scores = topic_vector[top_indices]

        topic_df = pd.DataFrame({
            f'Topic {topic_idx+1} Word': words,
            f'Topic {topic_idx+1} Score': scores
        })
        all_topics_dfs.append(topic_df)

    final_df = pd.concat(all_topics_dfs, axis=1)
    try:
        final_df.to_csv(filepath, index=False, sep=',', float_format='%.5f', encoding='utf-8')
    except Exception as e:
        print(f"\n    ATTENTION : Échec sauvegarde CSV des topics ({filepath}): {e}")


In [23]:
# --- Exécution Principale ---
print("Démarrage de l'exploration NMF avec scénarios H & W...")
try:
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(topic_output_dir, exist_ok=True)
    os.makedirs(init_csv_dir, exist_ok=True)
    print(f"Vérification: Utilisation du dossier de sortie général: '{output_dir}'")
    print(f"              Dossier pour initialisations CSV: '{init_csv_dir}'")
    print(f"              Dossier pour détails des topics: '{topic_output_dir}'")
except OSError as e:
    print(f"\nERREUR CRITIQUE: Impossible de créer les dossiers de sortie dans '{output_dir}'. Vérifiez les permissions.")
    print(f"Erreur système: {e}")
    # Optionnel: arrêter le script si les dossiers sont indispensables
    # import sys
    # sys.exit(1)


Démarrage de l'exploration NMF avec scénarios H & W...


NameError: name 'output_dir' is not defined

In [ ]:
import time
import os
import itertools
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from tqdm.auto import tqdm # Use auto for better notebook/console detection

# --- NOUVEAUX IMPORTS (GENSIM) ---
# Assurez-vous que Gensim est installé: pip install gensim
try:
    from gensim.models.coherencemodel import CoherenceModel
    from gensim.corpora import Dictionary
    print("Bibliothèque GENSIM (CoherenceModel, Dictionary) trouvée.")
    GENSIM_AVAILABLE = True
except ImportError:
    print("\nATTENTION : La bibliothèque GENSIM n'a pas pu être importée.")
    print("Veuillez l'installer via 'pip install gensim'.")
    print("Le calcul de la cohérence c_npmi sera désactivé.")
    CoherenceModel = None
    Dictionary = None
    GENSIM_AVAILABLE = False


import numpy as np
from sklearn.decomposition import NMF
from sklearn.utils import check_random_state
# from sklearn.utils.extmath import safe_sparse_dot, squared_norm # Pas nécessaire ici
import time
import os
import itertools
import pandas as pd # Ajouté pour une gestion plus facile des résultats finaux
# Tqdm import was missing in the provided snippet, assuming it's needed
from tqdm.auto import tqdm

# --- Supposons que ces variables sont déjà définies ---
# ATTENTION: Les variables suivantes DOIVENT être définies dans une cellule précédente
# ou au début de ce script pour qu'il fonctionne :
# - tfidf: La matrice de données (np.ndarray ou sparse matrix, shape: n_samples, n_features)
# - tfidf_feature_names: La liste ou array des noms de features/mots (longueur: n_features)
#
# Exemple de déclaration (si elles n'existent pas vraiment avant):
# n_samples_in_tfidf = 100
# n_features_in_tfidf = 500
# np.random.seed(42)
# tfidf = np.random.rand(n_samples_in_tfidf, n_features_in_tfidf) * 10
# tfidf[tfidf < 0] = 0
# tfidf_feature_names = np.array([f"mot_{i}" for i in range(n_features_in_tfidf)], dtype=object)
# print(f"Utilisation de données simulées: tfidf shape={tfidf.shape}, feature_names len={len(tfidf_feature_names)}")


alpha_W = 0.0 # Fourni
alpha_H = 0.0 # Fourni
l1_ratio = 0.5 # Ratio L1/L2 pour la régularisation (peut être fourni aussi)
# --- Fin des variables supposées définies ---


# --- Configuration ---
topic_list = [7, 12, 15] # EXEMPLE: Tester pour 7, 12 et 15 topics

# Scénarios de décroissance pour H (sur les items/mots)
h_decay_powers = [0.7, 1.0, 1.5]
h_num_items_decay = 50          # Nb items pour la décroissance dans H
h_noise_level = 0.15            # Bruit/variation pour H

# Scénarios de décroissance pour W (sur les topics/composantes)
w_decay_powers = [0.7, 1.0, 1.5]
w_noise_level = 0.15            # Bruit/variation pour W

# Paramètres NMF et autres
max_iter = 10000                 # Max iterations pour solveur NMF
random_state_solver = 1         # Graine pour le solveur NMF (si applicable)
random_state_init = 42          # Graine pour la génération des initialisations custom H et W
random_state_nndsvd = 1         # Graine spécifique pour la comparaison NNDSVD
epsilon = 1e-6                  # Petite valeur pour éviter les zéros
output_dir = "/app/nmf_results" # Dossier général pour tous les résultats
topic_output_dir = os.path.join(output_dir, "topic_details") # Sous-dossier pour les CSV de topics
init_csv_dir = os.path.join(output_dir, "initializations_csv") # Sous-dossier pour les CSV d'initialisation


# --- Assurez-vous que ces variables sont définies AVANT ---
# Exemple de valeurs par défaut, REMPLACEZ par vos vraies valeurs
# topic_list = [5, 10, 15]
# h_decay_powers = [0.5, 1.0]
# w_decay_powers = [0.0, 0.5]
# tfidf = ... # Votre matrice TF-IDF (sparse ou dense)
# tfidf_feature_names = ... # Liste/Array des noms de features (mots)
# tokenized_documents = ... # LISTE DE LISTES DE TOKENS (ex: [['mot1', 'mot2'], ['mot3', 'mot1']])
# create_decreasing_H = ... # Votre fonction
# create_decreasing_W = ... # Votre fonction
# save_topics_to_file = ... # Votre fonction
# h_num_items_decay = 10
# h_noise_level = 0.01
# w_noise_level = 0.01

alpha_W = 0.0 # Ou votre valeur de régularisation
alpha_H = 0.0 # Ou votre valeur de régularisation
l1_ratio = 0.0 # Ou votre valeur

n_top_words = 10 # Nombre de mots clés à extraire/sauvegarder ET utiliser pour définir les topics pour Gensim
COHERENCE_WINDOW_SIZE = 30 # Fenêtre pour le calcul de cohérence NPMI avec Gensim

# --- Vérifications initiales des prérequis ---
required_vars = ['topic_list', 'h_decay_powers', 'w_decay_powers', 'tfidf',
                 'tfidf_feature_names', 'tokenized_documents', #'init_csv_dir', # Optionnel
                 'output_dir', 'topic_output_dir', 'create_decreasing_H',
                 'create_decreasing_W', 'save_topics_to_file', 'n_top_words']
for var in required_vars:
    if var not in globals():
        raise NameError(f"Variable essentielle '{var}' n'est pas définie.")

if not GENSIM_AVAILABLE:
     print("\nATTENTION: Gensim n'est pas chargé. Le calcul de la cohérence sera désactivé.")
else:
    # Vérification spécifique pour Gensim: tokenized_documents doit être list[list[str]]
    if 'tokenized_documents' not in globals():
         raise NameError("Variable essentielle 'tokenized_documents' non définie (requise pour Gensim).")
    if not isinstance(tokenized_documents, list) or not all(isinstance(doc, list) for doc in tokenized_documents):
        raise TypeError("`tokenized_documents` doit être une liste de listes de strings pour GENSIM.")
    # Vérification cohérence tokenized_documents et tfidf (nombre de documents)
    if 'tfidf' in globals():
        if len(tokenized_documents) != tfidf.shape[0]:
             print(f"ATTENTION: Le nombre de documents dans 'tokenized_documents' ({len(tokenized_documents)}) "
                   f"ne correspond pas au nombre de lignes dans 'tfidf' ({tfidf.shape[0]}). "
                   f"La cohérence Gensim pourrait être basée sur un nombre différent de textes.")

# --- Création du Dictionnaire Gensim (une seule fois) ---
gensim_dictionary = None
if GENSIM_AVAILABLE:
    print("\nCréation du dictionnaire Gensim à partir de tokenized_documents...")
    try:
        gensim_dictionary = Dictionary(tokenized_documents)
        # Optionnel: Filtrer
        # gensim_dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)
        print(f"Dictionnaire Gensim créé avec {len(gensim_dictionary)} tokens uniques.")
    except Exception as e:
        print(f"\nERREUR lors de la création du dictionnaire Gensim : {e}")
        print("Le calcul de la cohérence c_npmi via Gensim sera désactivé.")
        GENSIM_AVAILABLE = False # Désactiver si le dictionnaire échoue
        CoherenceModel = None
        Dictionary = None


# --- Fonctions d'aide (Extraction + Cohérence Gensim) ---

# --- Initialisation des variables de suivi ---
overall_best_error = float('inf')
overall_best_params = {}
all_results_list = []

# Dictionnaires pour stocker les résultats finaux par K
best_custom_H_matrices = {}
nndsvd_H_matrices = {}
best_custom_models = {} # Stocke les objets model custom (celui avec la meilleure erreur)
nndsvd_models = {}      # Stocke les objets model NNDSVD
best_custom_coherence = {} # Stocker la cohérence du meilleur modèle custom (par erreur) par K
nndsvd_coherence = {} # Stocker la cohérence NNDSVD par K

# --- Boucle sur le nombre de topics ---
for num_topic in tqdm(topic_list, desc="PROCESSUS GLOBAL (Nb Topics)"):

    print(f"\n===== Test pour num_topic = {num_topic} =====")
    n_samples, n_features = tfidf.shape # Récupérer dimensions

    # Vérifier tfidf_feature_names (cohérence) - déjà fait mais bon à garder ici aussi
    if len(tfidf_feature_names) != n_features:
         print(f"ATTENTION K={num_topic}: Longueur 'tfidf_feature_names' ({len(tfidf_feature_names)}) "
               f"!= nb features tfidf ({n_features}). Extraction de mots échouera.")
         continue # Sauter ce K si les features ne correspondent pas

    scenario_combinations = list(itertools.product(h_decay_powers, w_decay_powers))

    # Variables pour suivre le meilleur run custom pour CE num_topic
    current_best_model_for_topic = None
    current_best_error_for_topic = float('inf')
    current_best_coherence_for_topic = -float('inf') # Init à -inf car on maximise cohérence (mais on stocke celle du meilleur par erreur)
    current_best_scenario_name_for_topic = None

    pbar_scenarios = tqdm(scenario_combinations, desc=f"  Scénarios Custom (K={num_topic})", leave=False, unit="scenario")
    for h_power, w_power in pbar_scenarios:

        scenario_name = f"H_p={h_power:.2f}_W_p={w_power:.2f}"
        pbar_scenarios.set_postfix_str(scenario_name, refresh=True)

        # 1. Créer les matrices d'initialisation
        current_H_init, current_W_init = None, None
        init_creation_success = False
        try:
            current_H_init = create_decreasing_H(
                num_topic, n_features, h_num_items_decay, h_power, h_noise_level, random_state_init, epsilon
            )
            current_W_init = create_decreasing_W(
                n_samples, num_topic, w_power, w_noise_level, random_state_init, epsilon
            )
            # Vérifier les dimensions AVANT de lancer NMF
            if current_W_init.shape != (n_samples, num_topic) or current_H_init.shape != (num_topic, n_features):
                 raise ValueError(f"Dims init custom INCOHÉRENTES: W={current_W_init.shape} (attendu {(n_samples, num_topic)}), H={current_H_init.shape} (attendu {(num_topic, n_features)})")
            init_creation_success = True
        except Exception as e:
            print(f"\n    ERREUR création init pour {scenario_name} (K={num_topic}): {e}")

        # 2. Sauvegarder les matrices d'initialisation (Optionnel - désactivé ici)
        # if init_creation_success and init_csv_dir:
        #     # ... votre code de sauvegarde CSV ...
        #     pass

        # 3. Exécuter NMF
        model = None
        error = float('inf')
        coherence_score = np.nan # Initialiser score cohérence
        duration = 0
        success = False
        topic_words_custom = [] # Initialiser liste de mots

        if init_creation_success:
            model = NMF(
                n_components=num_topic, init='custom', solver='cd',
                random_state=random_state_solver, max_iter=max_iter,
                alpha_W=alpha_W, alpha_H=alpha_H, l1_ratio=l1_ratio,
                # tol=1e-4 # Peut aider parfois
            )
            start_time = time.time()
            try:
                # NMF fit avec les matrices fournies
                model.fit(tfidf, W=current_W_init.copy(), H=current_H_init.copy())
                error = model.reconstruction_err_
                duration = time.time() - start_time
                success = True

                # --- NOUVEAU: Calcul de la cohérence GENSIM APRÈS fit réussi ---
                if success:
                    #print(f"    NMF Custom {scenario_name} K={num_topic} réussi. Calcul cohérence (Gensim)...")
                    # Extraire les mots clés
                    topic_words_custom = extract_top_words(model.components_, tfidf_feature_names, n_top_words)
                    if topic_words_custom and GENSIM_AVAILABLE: # Si extraction ok ET Gensim dispo
                        # Calculer la cohérence avec Gensim
                         coherence_score = calculate_coherence_gensim( # Utiliser la nouvelle fonction
                             topic_words_custom,
                             tokenized_documents,
                             gensim_dictionary,         # Passer le dictionnaire
                             COHERENCE_WINDOW_SIZE      # Passer la window size
                             # measure='c_npmi' # est la valeur par défaut dans la fonction
                         )
                         #print(f"    -> Cohérence C_NPMI (Gensim): {coherence_score:.4f}") # Log optionnel ici, fait dans comparaison
                    elif not topic_words_custom:
                        print(f"    WARN: Échec extraction mots pour cohérence (custom {scenario_name} K={num_topic}).")
                    # Si Gensim non dispo, coherence_score reste NaN (initialisé)

            except ValueError as ve:
                 # Gère spécifiquement les erreurs de dimensions ou autres ValueError de NMF/fit
                 print(f"\n    ERREUR (ValueError) NMF {scenario_name} (K={num_topic}): {ve}")
                 duration = time.time() - start_time
                 success = False
            except Exception as e:
                print(f"\n    ERREUR (Autre) NMF {scenario_name} (K={num_topic}): {e}")
                duration = time.time() - start_time
                success = False

        # 4. Stocker les résultats du run (erreur ET cohérence Gensim)
        result_data = {
            'num_topic': num_topic, 'h_power': h_power, 'w_power': w_power,
            'scenario': scenario_name,
            'error': error if success else np.nan,
            'coherence_npmi_gensim': coherence_score, # Nom de colonne mis à jour
            'duration': duration, 'success': success
        }
        all_results_list.append(result_data)

        # 5. Mettre à jour le meilleur modèle pour ce `num_topic` (basé sur l'ERREUR)
        if success and error < current_best_error_for_topic:
            # print(f"    -> Nouveau meilleur custom pour K={num_topic} (par erreur): {scenario_name} (Err: {error:.4f}, Coh(Gensim): {coherence_score:.4f})") # Log optionnel
            current_best_error_for_topic = error
            current_best_coherence_for_topic = coherence_score # Stocker aussi sa cohérence
            current_best_model_for_topic = model # Stocker l'objet modèle
            current_best_scenario_name_for_topic = scenario_name

        # 6. Mettre à jour le meilleur modèle global (basé sur l'ERREUR)
        if success and error < overall_best_error:
            overall_best_error = error
            overall_best_params = {
                'num_topic': num_topic, 'h_power': h_power, 'w_power': w_power,
                'scenario': scenario_name, 'error': error,
                'coherence_npmi_gensim': coherence_score, # Nom de colonne mis à jour
                'duration': duration,
                'h_num_items_decay': h_num_items_decay, 'h_noise_level': h_noise_level,
                'w_noise_level': w_noise_level,
                'random_state_init': random_state_init, 'random_state_solver': random_state_solver,
                'alpha_W': alpha_W, 'alpha_H': alpha_H, 'l1_ratio': l1_ratio, 'max_iter': max_iter
            }

    # --- Fin de la boucle sur les combinaisons H/W pour un K donné ---
    if current_best_scenario_name_for_topic:
        # Récupérer le meilleur modèle stocké et sa cohérence
        best_model = current_best_model_for_topic
        best_coherence = current_best_coherence_for_topic
        print(f"  Meilleur scénario custom pour K={num_topic} (par erreur): '{current_best_scenario_name_for_topic}' "
              f"(Erreur: {current_best_error_for_topic:.4f}, Cohérence(Gensim): {best_coherence:.4f})")
        # Stocker le meilleur modèle, sa matrice H, et sa cohérence DÉFINITIVEMENT pour ce K
        if best_model is not None:
            best_custom_models[num_topic] = best_model
            best_custom_H_matrices[num_topic] = best_model.components_.copy()
            best_custom_coherence[num_topic] = best_coherence # Stocker la cohérence associée
    else:
        print(f"  Aucun run NMF custom réussi pour K={num_topic} avec les initialisations testées.")
        best_custom_coherence[num_topic] = np.nan # Marquer comme non calculé/échoué

# --- Fin de la boucle sur num_topic (Initialisations custom) ---


# --- Convertir la liste des résultats en DataFrame pandas ---
results_df = pd.DataFrame(all_results_list)

# Récupérer les meilleures erreurs custom pour la comparaison directe
if not results_df.empty and 'error' in results_df.columns:
    best_custom_errors = results_df[results_df['success']].groupby('num_topic')['error'].min().to_dict()
else:
    best_custom_errors = {}
# Note: best_custom_coherence[k] contient déjà la cohérence Gensim du modèle ayant la meilleure ERREUR pour K=k.


# --- NOUVELLE ÉTAPE : Comparaison avec NNDSVD (avec calcul cohérence Gensim) ---
print("\n===== Comparaison avec l'initialisation NNDSVD (Cohérence via Gensim) =====")
nndsvd_results = {} # Stocke les infos complètes par K pour NNDSVD

pbar_nndsvd = tqdm(topic_list, desc="  NMF NNDSVD (par K)", leave=False, unit="K")
for num_topic in pbar_nndsvd:
    pbar_nndsvd.set_postfix_str(f"K={num_topic}", refresh=True)

    n_samples, n_features = tfidf.shape # Répété pour clarté

    model_nndsvd = NMF(
        n_components=num_topic, init='nndsvd', random_state=random_state_nndsvd,
        solver='cd', max_iter=max_iter, alpha_W=alpha_W, alpha_H=alpha_H, l1_ratio=l1_ratio
        # tol=1e-4
    )

    start_time_nndsvd = time.time()
    success_nndsvd = False
    error_nndsvd = float('inf')
    coherence_nndsvd = np.nan # Initialiser cohérence NNDSVD
    duration_nndsvd = 0
    topic_words_nndsvd = []

    try:
        # print(f"  Running NNDSVD for K={num_topic}...") # Moins verbeux
        model_nndsvd.fit(tfidf)
        error_nndsvd = model_nndsvd.reconstruction_err_
        duration_nndsvd = time.time() - start_time_nndsvd
        success_nndsvd = True
        # print(f"    NNDSVD K={num_topic} réussi. Calcul cohérence (Gensim)...") # Moins verbeux

        # Stocker le modèle et H si succès
        nndsvd_models[num_topic] = model_nndsvd
        nndsvd_H_matrices[num_topic] = model_nndsvd.components_.copy()

        # --- NOUVEAU: Calcul cohérence GENSIM pour NNDSVD ---
        if success_nndsvd:
             # Extraire les mots clés
             topic_words_nndsvd = extract_top_words(model_nndsvd.components_, tfidf_feature_names, n_top_words)
             if topic_words_nndsvd and GENSIM_AVAILABLE: # Si extraction ok ET Gensim dispo
                 # Calculer la cohérence Gensim
                 coherence_nndsvd = calculate_coherence_gensim( # Utiliser la nouvelle fonction
                     topic_words_nndsvd,
                     tokenized_documents,
                     gensim_dictionary,         # Passer le dictionnaire
                     COHERENCE_WINDOW_SIZE      # Passer la window size
                 )
                 # print(f"    -> Cohérence C_NPMI (Gensim): {coherence_nndsvd:.4f}") # Log optionnel
             elif not topic_words_nndsvd:
                 print(f"    WARN: Échec extraction mots pour cohérence (NNDSVD K={num_topic}).")
             # Si Gensim non dispo, coherence_nndsvd reste NaN

    except Exception as e:
        print(f"\n    ERREUR NMF NNDSVD pour K={num_topic}: {e}")
        duration_nndsvd = time.time() - start_time_nndsvd # Durée même si erreur
        success_nndsvd = False

    # Stocker les résultats NNDSVD (incluant cohérence Gensim)
    nndsvd_results[num_topic] = {
        'error': error_nndsvd if success_nndsvd else np.nan,
        'coherence_npmi_gensim': coherence_nndsvd, # Nom de colonne mis à jour
        'duration': duration_nndsvd,
        'success': success_nndsvd
    }
    # Aussi stocker séparément pour accès facile dans la synthèse
    nndsvd_coherence[num_topic] = coherence_nndsvd


    # Afficher la comparaison immédiate (erreur et cohérence Gensim)
    best_custom_err_k = best_custom_errors.get(num_topic, float('inf'))
    best_custom_coh_k = best_custom_coherence.get(num_topic, np.nan) # Récupérer cohérence du meilleur custom (par erreur)

    print(f"  Comparaison K={num_topic}:")
    err_nndsvd_str = f"{error_nndsvd:.4f}" if success_nndsvd else "ÉCHEC"
    coh_nndsvd_str = f"{coherence_nndsvd:.4f}" if not np.isnan(coherence_nndsvd) else "N/A"
    print(f"    NNDSVD (rs={random_state_nndsvd}): Err={err_nndsvd_str}, Coh(Gensim)={coh_nndsvd_str} (Dur: {duration_nndsvd:.2f}s)")

    if not np.isnan(best_custom_err_k) and best_custom_err_k != float('inf'):
        best_custom_err_str = f"{best_custom_err_k:.4f}"
        best_custom_coh_str = f"{best_custom_coh_k:.4f}" if not np.isnan(best_custom_coh_k) else "N/A"
        print(f"    Best Custom(err): Err={best_custom_err_str}, Coh(Gensim)={best_custom_coh_str}")
        # Comparaison basée sur l'erreur
        if success_nndsvd and best_custom_err_k < error_nndsvd:
             print(f"    -> Meilleur Custom supérieur (par erreur).")
        elif success_nndsvd:
             print(f"    -> NNDSVD supérieur ou égal (par erreur).")
        elif not success_nndsvd: # NNDSVD échoue, Custom réussit
             print(f"    -> Meilleur Custom a réussi là où NNDSVD a échoué.")
    else: # Aucun run custom réussi pour ce K
        if success_nndsvd:
            print(f"    (NNDSVD a réussi, aucun run custom réussi pour comparer)")
        else: # Aucun des deux n'a réussi
            print(f"    (Ni Custom ni NNDSVD n'ont réussi pour K={num_topic})")


# --- Synthèse Finale ---
print("\n===== Synthèse Globale des Initialisations Custom (Cohérence via Gensim) =====")

# Afficher le DataFrame des résultats custom (avec cohérence Gensim)
print("\n--- Tableau Récapitulatif des Résultats Custom ---")
if not results_df.empty:
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.max_columns', None)
    # Trier par topic, puis par erreur (ou cohérence Gensim si vous préférez)
    results_df_sorted = results_df.sort_values(by=['num_topic', 'error'], ascending=[True, True])
    # results_df_sorted = results_df.sort_values(by=['num_topic', 'coherence_npmi_gensim'], ascending=[True, False]) # Tri par cohérence (décroissant)
    print(results_df_sorted.to_string(index=False, float_format='%.4f', na_rep='N/A'))

    # Sauvegarder le DataFrame en CSV
    try:
        # Mettre à jour le nom de fichier pour refléter l'utilisation de Gensim
        results_filename = os.path.join(output_dir, "nmf_all_scenario_results_gensim_coherence.csv")
        results_df_sorted.to_csv(results_filename, index=False, sep=',', float_format='%.8f', na_rep='NaN')
        print(f"\nTableau des résultats custom sauvegardé dans : '{results_filename}'")
    except Exception as e:
        print(f"\nATTENTION : Échec sauvegarde CSV des résultats custom : {e}")
else:
    print("Aucun résultat à afficher ou sauvegarder pour les initialisations custom.")


# Afficher les paramètres du meilleur run global custom (basé sur l'ERREUR)
print("\n--- Meilleur Résultat Global Custom Trouvé (basé sur l'ERREUR de reconstruction) ---")
if overall_best_params:
    print("Paramètres correspondants :")
    for key, val in overall_best_params.items():
        if isinstance(val, float): print(f"  {key}: {val:.4f}")
        else: print(f"  {key}: {val}")
    # Note: overall_best_params contient aussi 'coherence_npmi_gensim'
else:
    print("Aucun run NMF custom réussi trouvé sur l'ensemble des configurations testées.")


# Rappel de la comparaison NNDSVD (avec cohérence Gensim)
print(f"\n--- Rappel Comparaison Finale NNDSVD vs Meilleur Custom (par K, Cohérence Gensim w={COHERENCE_WINDOW_SIZE}) ---")
if nndsvd_results:
    print(f"(NNDSVD: rs={random_state_nndsvd}. Best Custom: basé sur min ERREUR pour ce K)")
    # Ajustement des largeurs de colonnes pour la cohérence
    print(f"{'K':<4} | {'NNDSVD Err':<12} | {'NNDSVD Coh(G)':<14} | {'Best Cust Err':<15} | {'Best Cust Coh(G)':<17}")
    print("-" * (4 + 12 + 14 + 15 + 17 + 10)) # Ajuster la longueur du séparateur
    for k in topic_list: # Itérer sur topic_list pour garder l'ordre
        # Utiliser les résultats stockés pour NNDSVD
        res_nndsvd = nndsvd_results.get(k, {'success': False, 'error': np.nan, 'coherence_npmi_gensim': np.nan})
        # Utiliser les résultats stockés pour le meilleur Custom (par erreur)
        best_cust_err = best_custom_errors.get(k, np.nan)
        best_cust_coh = best_custom_coherence.get(k, np.nan) # Cohérence Gensim associée

        # Formatage des chaînes de sortie
        nndsvd_err_str = f"{res_nndsvd['error']:.4f}" if res_nndsvd['success'] else "ÉCHEC"
        nndsvd_coh_str = f"{res_nndsvd['coherence_npmi_gensim']:.4f}" if not np.isnan(res_nndsvd['coherence_npmi_gensim']) else "N/A"
        cust_err_str = f"{best_cust_err:.4f}" if not np.isnan(best_cust_err) else "N/A"
        cust_coh_str = f"{best_cust_coh:.4f}" if not np.isnan(best_cust_coh) else "N/A"

        # Affichage aligné
        print(f"{k:<4} | {nndsvd_err_str:<12} | {nndsvd_coh_str:<14} | {cust_err_str:<15} | {cust_coh_str:<17}")

else:
    print("Aucun résultat NNDSVD n'a été calculé (la boucle a peut-être échoué).")


# --- Sauvegarde des Matrices H Formatées (Top Mots) ---
# Cette partie reste identique car elle dépend des matrices H stockées et de tfidf_feature_names
print(f"\n===== Sauvegarde des Détails des Topics (Top {n_top_words} Mots) =====")
print(f"Les fichiers seront sauvegardés dans : '{topic_output_dir}'")

# Vérification finale avant sauvegarde (devrait être OK si le script est arrivé ici)
feature_names_ok = ('tfidf_feature_names' in globals() and
                    'tfidf' in globals() and
                    len(tfidf_feature_names) == tfidf.shape[1])

if not feature_names_ok:
    print("Sauvegarde des détails des topics ANNULÉE en raison d'une incohérence détectée "
          "entre tfidf et tfidf_feature_names.")
else:
    local_feature_names = tfidf_feature_names # Utilisation sûre
    os.makedirs(topic_output_dir, exist_ok=True) # Assurer que le dossier existe

    for k in topic_list:
        print(f"\n--- Sauvegarde Topics pour K={k} ---")
        saved_custom = False
        saved_nndsvd = False

        # Sauvegarde meilleur custom pour K (basé sur erreur)
        if k in best_custom_H_matrices:
            H_custom = best_custom_H_matrices[k]
            filepath_custom = os.path.join(topic_output_dir, f"topics_K{k}_best_custom_by_error.csv")
            try:
                # Assurez-vous que save_topics_to_file est bien définie
                save_topics_to_file(H_custom, local_feature_names, filepath_custom, n_top_words)
                print(f"  -> Fichier Best Custom (par erreur) : '{os.path.basename(filepath_custom)}'")
                saved_custom = True
            except NameError:
                 print("  ERREUR: La fonction 'save_topics_to_file' n'est pas définie!")
                 break # Inutile de continuer si la fonction manque
            except Exception as e:
                print(f"  ERREUR sauvegarde topics best custom K={k}: {e}")
        else:
            print("  (Pas de matrice H best custom trouvée pour K={k})")

        # Sauvegarde NNDSVD pour K
        if k in nndsvd_H_matrices:
            H_nndsvd = nndsvd_H_matrices[k]
            filepath_nndsvd = os.path.join(topic_output_dir, f"topics_K{k}_nndsvd_rs{random_state_nndsvd}.csv")
            try:
                 # Assurez-vous que save_topics_to_file est bien définie
                save_topics_to_file(H_nndsvd, local_feature_names, filepath_nndsvd, n_top_words)
                print(f"  -> Fichier NNDSVD                 : '{os.path.basename(filepath_nndsvd)}'")
                saved_nndsvd = True
            except NameError:
                 print("  ERREUR: La fonction 'save_topics_to_file' n'est pas définie!")
                 break # Inutile de continuer si la fonction manque
            except Exception as e:
                print(f"  ERREUR sauvegarde topics NNDSVD K={k}: {e}")
        else:
            print(f"  (Pas de matrice H NNDSVD trouvée pour K={k})")

        if not saved_custom and not saved_nndsvd:
                print("  (Aucune matrice H à sauvegarder pour ce K)")


# --- Calcul et Affichage des Poids Totaux des Topics et CV ---
# Cette partie reste identique car elle utilise les modèles stockés
print("\n===== Poids Totaux des Topics et Coefficient de Variation (CV) =====")

if 'tfidf' not in globals():
     print("\nATTENTION CRITIQUE: Variable `tfidf` non définie. Calculs de W impossibles.")
else:
    # Utiliser les modèles stockés
    if 'best_custom_models' not in globals(): best_custom_models = {}
    if 'nndsvd_models' not in globals(): nndsvd_models = {}

    for k in topic_list:
        print(f"\n--- Statistiques Poids pour K={k} ---")
        stats_calculated = False

        # Stats pour le meilleur custom (par erreur)
        if k in best_custom_models:
            model_custom = best_custom_models[k]
            try:
                W_custom = model_custom.transform(tfidf)
                custom_topic_weights = W_custom.sum(axis=0)
                mean_weight = np.mean(custom_topic_weights)
                cv_custom = np.std(custom_topic_weights) / mean_weight if mean_weight > 1e-9 else 0.0

                print("  Statistiques Best Custom (par erreur) :")
                weights_str = [f"{w:.3f}" for w in custom_topic_weights]
                print(f"    Poids Totaux par Topic      : [{', '.join(weights_str)}]")
                print(f"    Coefficient de Variation    : {cv_custom:.3f}")
                stats_calculated = True
            except Exception as e:
                print(f"    Erreur calcul/transform W stats pour best custom (K={k}): {e}")
        else:
            print("  (Pas de modèle 'best custom' trouvé pour K={k})")

        # Stats pour NNDSVD
        if k in nndsvd_models:
            model_nndsvd = nndsvd_models[k]
            try:
                W_nndsvd = model_nndsvd.transform(tfidf)
                nndsvd_topic_weights = W_nndsvd.sum(axis=0)
                mean_weight = np.mean(nndsvd_topic_weights)
                cv_nndsvd = np.std(nndsvd_topic_weights) / mean_weight if mean_weight > 1e-9 else 0.0

                print("  Statistiques NNDSVD                 :")
                weights_str = [f"{w:.3f}" for w in nndsvd_topic_weights]
                print(f"    Poids Totaux par Topic      : [{', '.join(weights_str)}]")
                print(f"    Coefficient de Variation    : {cv_nndsvd:.3f}")
                stats_calculated = True
            except Exception as e:
                print(f"    Erreur calcul/transform W stats pour NNDSVD (K={k}): {e}")
        else:
            print(f"  (Pas de modèle NNDSVD trouvé pour K={k})")

        if not stats_calculated:
            print("  (Aucun modèle disponible pour calculer les statistiques de poids pour ce K)")

print("\nCalcul des statistiques des topics terminé.")
print("\nExploration NMF complète terminée (Cohérence via Gensim).")

Bibliothèque GENSIM (CoherenceModel, Dictionary) trouvée.

Création du dictionnaire Gensim à partir de tokenized_documents...
Dictionnaire Gensim créé avec 61062 tokens uniques.


PROCESSUS GLOBAL (Nb Topics):   0%|          | 0/3 [00:00<?, ?it/s]


===== Test pour num_topic = 7 =====


  Scénarios Custom (K=7):   0%|          | 0/9 [00:00<?, ?scenario/s]

  Meilleur scénario custom pour K=7 (par erreur): 'H_p=1.00_W_p=1.00' (Erreur: 8675.5236, Cohérence(Gensim): 0.1040)

===== Test pour num_topic = 12 =====


...

  Meilleur scénario custom pour K=12 (par erreur): 'H_p=1.00_W_p=1.50' (Erreur: 8627.0289, Cohérence(Gensim): 0.0914)

===== Test pour num_topic = 15 =====


  Scénarios Custom (K=15):   0%|          | 0/9 [00:00<?, ?scenario/s]

  Meilleur scénario custom pour K=15 (par erreur): 'H_p=1.00_W_p=0.70' (Erreur: 8602.2419, Cohérence(Gensim): 0.0851)

===== Comparaison avec l'initialisation NNDSVD (Cohérence via Gensim) =====


  NMF NNDSVD (par K):   0%|          | 0/3 [00:00<?, ?K/s]

  Comparaison K=7:
    NNDSVD (rs=1): Err=8675.5232, Coh(Gensim)=0.1040 (Dur: 2.52s)
    Best Custom(err): Err=8675.5236, Coh(Gensim)=0.1040
    -> NNDSVD supérieur ou égal (par erreur).
  Comparaison K=12:
    NNDSVD (rs=1): Err=8627.0285, Coh(Gensim)=0.0914 (Dur: 5.46s)
    Best Custom(err): Err=8627.0289, Coh(Gensim)=0.0914
    -> NNDSVD supérieur ou égal (par erreur).
  Comparaison K=15:
    NNDSVD (rs=1): Err=8602.0467, Coh(Gensim)=0.0917 (Dur: 7.53s)
    Best Custom(err): Err=8602.2419, Coh(Gensim)=0.0851
    -> NNDSVD supérieur ou égal (par erreur).

===== Synthèse Globale des Initialisations Custom (Cohérence via Gensim) =====

--- Tableau Récapitulatif des Résultats Custom ---
 num_topic  h_power  w_power          scenario     error  coherence_npmi_gensim  duration  success
         7   1.0000   1.0000 H_p=1.00_W_p=1.00 8675.5236                 0.1040    2.6801     True
         7   1.5000   1.0000 H_p=1.50_W_p=1.00 8675.5236                 0.1040    4.3266     True
        

In [ ]:
# 1. Créer les matrices d'initialisation
current_H_init, current_W_init = None, None
init_creation_success = False
try:
    current_H_init = create_decreasing_H(
        num_topic, n_features, h_num_items_decay, h_power, h_noise_level, 42, epsilon
    )
    current_W_init = create_decreasing_W(
        n_samples, num_topic, w_power, w_noise_level, 42, epsilon
    )
    # Vérifier les dimensions AVANT de lancer NMF
    if current_W_init.shape != (n_samples, num_topic) or current_H_init.shape != (num_topic, n_features):
            raise ValueError(f"Dims init custom INCOHÉRENTES: W={current_W_init.shape} (attendu {(n_samples, num_topic)}), H={current_H_init.shape} (attendu {(num_topic, n_features)})")
    init_creation_success = True
except Exception as e:
    print(f"\n    ERREUR création init pour {scenario_name} (K={num_topic}): {e}")

# 2. Sauvegarder les matrices d'initialisation (Optionnel - désactivé ici)
# if init_creation_success and init_csv_dir:
#     # ... votre code de sauvegarde CSV ...
#     pass

# 3. Exécuter NMF
model = None
error = float('inf')
coherence_score = np.nan # Initialiser score cohérence
duration = 0
success = False
topic_words_custom = [] # Initialiser liste de mots

if init_creation_success:
    model = NMF(
        n_components=num_topic, init='custom', solver='cd',
        random_state=1, max_iter=max_iter,
        alpha_W=alpha_W, alpha_H=alpha_H, l1_ratio=l1_ratio,
        # tol=1e-4 # Peut aider parfois
    )
    start_time = time.time()
    try:
        # NMF fit avec les matrices fournies
        model.fit(tfidf, W=current_W_init.copy(), H=current_H_init.copy())
        error = model.reconstruction_err_
        duration = time.time() - start_time
        success = True

        # --- NOUVEAU: Calcul de la cohérence GENSIM APRÈS fit réussi ---
        if success:
            #print(f"    NMF Custom {scenario_name} K={num_topic} réussi. Calcul cohérence (Gensim)...")
            # Extraire les mots clés
            topic_words_custom = extract_top_words(model.components_, tfidf_feature_names, n_top_words)
            if topic_words_custom and GENSIM_AVAILABLE: # Si extraction ok ET Gensim dispo
                # Calculer la cohérence avec Gensim
                    coherence_score = calculate_coherence_gensim( # Utiliser la nouvelle fonction
                        topic_words_custom,
                        tokenized_documents,
                        gensim_dictionary,         # Passer le dictionnaire
                        COHERENCE_WINDOW_SIZE      # Passer la window size
                        # measure='c_npmi' # est la valeur par défaut dans la fonction
                    )
                    #print(f"    -> Cohérence C_NPMI (Gensim): {coherence_score:.4f}") # Log optionnel ici, fait dans comparaison
            elif not topic_words_custom:
                print(f"    WARN: Échec extraction mots pour cohérence (custom {scenario_name} K={num_topic}).")
            # Si Gensim non dispo, coherence_score reste NaN (initialisé)


In [ ]:
# --- Imports présumés faits ---
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
# from scipy.optimize import linear_sum_assignment # Supposé non utilisé directement ici, mais peut l'être dans calculate_topic_stability
import time
import os
from tqdm.notebook import tqdm # Ou from tqdm import tqdm

# --- NOUVEAUX IMPORTS NÉCESSAIRES (GENSIM) ---
try:
    from gensim.models.coherencemodel import CoherenceModel
    from gensim.corpora import Dictionary
    print("Bibliothèque GENSIM pour la cohérence (CoherenceModel, Dictionary) trouvée.")
    GENSIM_AVAILABLE = True
except ImportError:
    print("\nATTENTION : La bibliothèque GENSIM (ou spécifiquement gensim.models.CoherenceModel")
    print("ou gensim.corpora.Dictionary) n'a pas pu être importée.")
    print("Assurez-vous que GENSIM est correctement installé (pip install gensim).")
    print("Le calcul de c_npmi sera désactivé.")
    CoherenceModel = None # Pour éviter les erreurs plus tard
    Dictionary = None
    GENSIM_AVAILABLE = False

# --- Variables et Fonctions présumées définies ---
# NECESSAIREMENT DEFINIS AVANT :
# - tfidf: np.ndarray ou sparse matrix (n_samples, n_features)
# - tfidf_feature_names: list ou np.ndarray de strings (n_features)
# - tokenized_documents: list[list[str]] (Corpus tokenisé, PRÉREQUIS CRUCIAL pour c_npmi avec Gensim)
# - alpha_W, alpha_H, l1_ratio: float (paramètres NMF)
# - h_num_items_decay, h_noise_level, w_noise_level: float/int (params init custom)
# - max_iter: int (paramètre NMF)
# - epsilon: float (pour init custom)
# - create_decreasing_H: function
# - create_decreasing_W: function
# - calculate_topic_stability: function (prend une liste de matrices H)
# ---

print(f"Début du test de robustesse ÉTENDU (avec c_NPMI via GENSIM) pour K=15 uniquement.")
print(f"Heure: {time.strftime('%H:%M:%S %Z')}, Lieu: Toulouse")

# --- Paramètres spécifiques pour ce test ---
NUM_TOPIC_TO_TEST = 15
N_ROBUSTNESS_RUNS = 20
ROBUSTNESS_SEEDS = list(range(N_ROBUSTNESS_RUNS))
FIXED_SOLVER_STATE_FOR_ROBUSTNESS = 1
N_TOP_WORDS_FOR_COHERENCE = 10  # Nombre de mots top à extraire pour DÉFINIR les topics passés à Gensim
WINDOW_SIZE_FOR_COHERENCE = 30 # Taille de la fenêtre glissante pour c_NPMI (Gensim)

# --- Choix du Scénario Custom Fixe ---
fixed_h_power_custom = 1.00
fixed_w_power_custom = 0.70
print(f"\nUtilisation du scénario custom fixe : K={NUM_TOPIC_TO_TEST}, h_power={fixed_h_power_custom}, w_power={fixed_w_power_custom}")
print(f"Nombre de runs de robustesse par méthode : {N_ROBUSTNESS_RUNS}")
print(f"Graine solveur fixe pour custom : {FIXED_SOLVER_STATE_FOR_ROBUSTNESS}")
print(f"Graines utilisées pour l'initialisation : {ROBUSTNESS_SEEDS[0]} à {ROBUSTNESS_SEEDS[-1]}")
if GENSIM_AVAILABLE:
    print(f"Paramètres c_NPMI (GENSIM) : topk={N_TOP_WORDS_FOR_COHERENCE}, window_size={WINDOW_SIZE_FOR_COHERENCE}")
else:
    print("Calcul de c_NPMI désactivé (GENSIM non trouvé ou import échoué).")

# --- Initialisation du stockage des résultats de robustesse ---
robustness_results = {
    'custom': {'errors': [], 'h_matrices': [], 'coherence_scores': []},
    'nndsvd': {'errors': [], 'h_matrices': [], 'coherence_scores': []}
}

# --- Vérifications Initiales Critiques ---
print("\nVérification des prérequis...")
required_vars = ['tfidf', 'tfidf_feature_names', 'alpha_W', 'alpha_H', 'l1_ratio',
                 'h_num_items_decay', 'h_noise_level', 'w_noise_level',
                 'max_iter', 'epsilon']
required_funcs = ['create_decreasing_H', 'create_decreasing_W', 'calculate_topic_stability']
# Ajout de la vérification pour tokenized_documents si Gensim est disponible
if GENSIM_AVAILABLE:
    required_vars.append('tokenized_documents')

missing_vars = [v for v in required_vars if v not in globals()]
missing_funcs = [f for f in required_funcs if f not in globals() or not callable(globals()[f])]

if missing_vars: raise NameError(f"Variables manquantes nécessaires : {', '.join(missing_vars)}")
if missing_funcs: raise NameError(f"Fonctions manquantes nécessaires : {', '.join(missing_funcs)}")
if GENSIM_AVAILABLE and 'tokenized_documents' not in globals():
     raise NameError("Variable manquante nécessaire pour GENSIM : tokenized_documents (list[list[str]])")
if GENSIM_AVAILABLE and 'tokenized_documents' in globals():
    if not isinstance(tokenized_documents, list) or not all(isinstance(doc, list) for doc in tokenized_documents):
        raise TypeError("`tokenized_documents` doit être une liste de listes de strings pour GENSIM.")

n_samples, n_features = tfidf.shape
if len(tfidf_feature_names) != n_features:
    raise ValueError(f"Incohérence de dimensions : {len(tfidf_feature_names)} feature_names vs {n_features} features dans tfidf.")
print("Prérequis vérifiés.")

# --- Préparation du Dictionnaire Gensim (une seule fois) ---
gensim_dictionary = None
if GENSIM_AVAILABLE:
    print("Création du dictionnaire Gensim à partir de tokenized_documents...")
    try:
        gensim_dictionary = Dictionary(tokenized_documents)
        # Optionnel: Filtrer les extrêmes si beaucoup de documents/vocabulaire
        # gensim_dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)
        print(f"Dictionnaire Gensim créé avec {len(gensim_dictionary)} tokens uniques.")
    except Exception as e:
        print(f"\nERREUR lors de la création du dictionnaire Gensim : {e}")
        print("Le calcul de la cohérence c_npmi via Gensim sera désactivé.")
        GENSIM_AVAILABLE = False # Désactiver si le dictionnaire échoue
        CoherenceModel = None
        Dictionary = None

# =============================================================================
# TEST DE ROBUSTESSE POUR K = NUM_TOPIC_TO_TEST
# =============================================================================
num_topic = NUM_TOPIC_TO_TEST

# --- Fonction utilitaire pour calculer la cohérence (version GENSIM) ---
def calculate_run_coherence_gensim(nmf_H, feature_names, tokenized_corpus, gensim_dictionary, n_top_words, window_size):
    """Calcule le score c_npmi pour une matrice H donnée en utilisant Gensim."""
    if not GENSIM_AVAILABLE or gensim_dictionary is None: # Ne rien faire si Gensim n'est pas dispo ou dictionnaire absent
        return np.nan

    try:
        # 1. Extraire les top words pour chaque topic à partir de nmf_H
        topics = []
        for topic_idx in range(nmf_H.shape[0]): # nmf_H.shape[0] == num_topic
            # Obtenir les indices des mots les plus importants pour ce topic
            top_word_indices = np.argsort(nmf_H[topic_idx, :])[-n_top_words:]
            # Récupérer les mots correspondants
            topic_words = [feature_names[i] for i in top_word_indices[::-1]] # Inverser pour avoir le plus important en premier
            topics.append(topic_words)

        # 2. Vérifier que les mots extraits sont dans le dictionnaire (Gensim peut échouer sinon)
        #    C'est une bonne pratique, bien que souvent non bloquant si les vocabulaires coïncident.
        vocab_present = all(all(word in gensim_dictionary.token2id for word in topic) for topic in topics)
        if not vocab_present:
             # Optionnel : loguer les mots manquants si besoin de débugger
             # missing_words = set(word for topic in topics for word in topic if word not in gensim_dictionary.token2id)
             # print(f"      Attention : Certains top words NMF ne sont pas dans le dictionnaire Gensim.") # Ignorer pour le moment
             pass # On tente quand même, Gensim gère parfois les mots absents

        # 3. Initialiser et calculer la métrique de cohérence Gensim
        cm = CoherenceModel(
            topics=topics,                 # La liste des listes de mots top
            texts=tokenized_corpus,        # Le corpus tokenisé (list[list[str]])
            dictionary=gensim_dictionary,  # Le dictionnaire Gensim
            coherence='c_npmi',            # La mesure de cohérence souhaitée
            window_size=window_size,       # La taille de la fenêtre glissante
            topn=n_top_words               # Gensim utilise aussi topn, ici redondant car on fournit 'topics'
                                           # mais on le garde cohérent.
        )
        coherence_score = cm.get_coherence()
        return coherence_score

    except Exception as e:
        # Afficher l'erreur spécifique de Gensim peut être utile
        print(f"\n      Erreur pendant le calcul de c_NPMI avec GENSIM : {e}")
        import traceback
        traceback.print_exc() # Imprime la trace complète pour aider au débogage
        return np.nan

# --- Robustesse Scénario A : Custom Fixe ---
print(f"\n--- Test Robustesse: Scénario Custom Fixe (h={fixed_h_power_custom}, w={fixed_w_power_custom}) ---")
pbar_robust_custom = tqdm(ROBUSTNESS_SEEDS, desc=f"  Robustesse Custom (K={num_topic})", leave=True, unit="seed")
for seed in pbar_robust_custom:
    pbar_robust_custom.set_postfix_str(f"seed={seed}", refresh=True)
    nmf_success = False
    current_H = None
    current_error = np.nan
    current_coherence = np.nan

    # 1. Créer W_init, H_init
    try:
        current_H_init = create_decreasing_H(
            num_topic, n_features, h_num_items_decay, fixed_h_power_custom,
            h_noise_level, seed, epsilon
        )
        current_W_init = create_decreasing_W(
            n_samples, num_topic, fixed_w_power_custom, w_noise_level,
            seed, epsilon
        )
        init_success = True
    except Exception as e:
        print(f"\n    Erreur création init custom (seed={seed}, K={num_topic}): {e}")
        init_success = False

    # 2. Exécuter NMF si init réussie
    if init_success:
        model_robust_custom = NMF(
            n_components=num_topic, init='custom', solver='cd',
            random_state=FIXED_SOLVER_STATE_FOR_ROBUSTNESS,
            max_iter=max_iter,
            alpha_W=alpha_W, alpha_H=alpha_H, l1_ratio=l1_ratio,
            # tol=1e-4 # Ajout d'une tolérance peut parfois aider la convergence
        )
        try:
            if current_W_init.shape != (n_samples, num_topic) or current_H_init.shape != (num_topic, n_features):
                 raise ValueError(f"Dims init custom R: W={current_W_init.shape}, H={current_H_init.shape} vs attendu ({n_samples},{num_topic}), ({num_topic},{n_features})")
            model_robust_custom.fit(tfidf, W=current_W_init.copy(), H=current_H_init.copy())
            current_error = model_robust_custom.reconstruction_err_
            current_H = model_robust_custom.components_.copy()
            nmf_success = True
        except Exception as e:
            print(f"\n    Erreur NMF custom (seed={seed}, K={num_topic}): {e}")
            nmf_success = False

    # 3. Calculer la cohérence si NMF réussie (avec GENSIM)
    if nmf_success and GENSIM_AVAILABLE:
         current_coherence = calculate_run_coherence_gensim( # Appel de la fonction version GENSIM
             current_H,
             tfidf_feature_names,
             tokenized_documents,
             gensim_dictionary,         # Passe le dictionnaire Gensim
             N_TOP_WORDS_FOR_COHERENCE, # Passe topk
             WINDOW_SIZE_FOR_COHERENCE  # Passe window_size
         )

    # 4. Stocker les résultats du run
    robustness_results['custom']['errors'].append(current_error)
    robustness_results['custom']['h_matrices'].append(current_H)
    robustness_results['custom']['coherence_scores'].append(current_coherence)

# --- Robustesse Scénario B : NNDSVD ---
print(f"\n--- Test Robustesse: NNDSVD ---")
pbar_robust_nndsvd = tqdm(ROBUSTNESS_SEEDS, desc=f"  Robustesse NNDSVD (K={num_topic})", leave=True, unit="seed")
for seed in pbar_robust_nndsvd:
    pbar_robust_nndsvd.set_postfix_str(f"seed={seed}", refresh=True)
    nmf_success = False
    current_H = None
    current_error = np.nan
    current_coherence = np.nan

    # 1. Exécuter NMF avec init NNDSVD
    model_robust_nndsvd = NMF(
        n_components=num_topic, init='nndsvd', solver='cd',
        random_state=seed,
        max_iter=max_iter,
        alpha_W=alpha_W, alpha_H=alpha_H, l1_ratio=l1_ratio,
        # tol=1e-4
    )
    try:
        model_robust_nndsvd.fit(tfidf)
        current_error = model_robust_nndsvd.reconstruction_err_
        current_H = model_robust_nndsvd.components_.copy()
        nmf_success = True
    except Exception as e:
        print(f"\n    Erreur NMF NNDSVD (seed={seed}, K={num_topic}): {e}")
        nmf_success = False

    # 2. Calculer la cohérence si NMF réussie (avec GENSIM)
    if nmf_success and GENSIM_AVAILABLE:
        current_coherence = calculate_run_coherence_gensim( # Appel de la fonction version GENSIM
            current_H,
            tfidf_feature_names,
            tokenized_documents,
            gensim_dictionary,         # Passe le dictionnaire Gensim
            N_TOP_WORDS_FOR_COHERENCE, # Passe topk
            WINDOW_SIZE_FOR_COHERENCE  # Passe window_size
        )

    # 3. Stocker les résultats du run
    robustness_results['nndsvd']['errors'].append(current_error)
    robustness_results['nndsvd']['h_matrices'].append(current_H)
    robustness_results['nndsvd']['coherence_scores'].append(current_coherence)


# =============================================================================
# ANALYSE ET AFFICHAGE DES RÉSULTATS DE ROBUSTESSE pour K=15
# =============================================================================
print(f"\n===== ANALYSE DE ROBUSTESSE FINALE POUR K={num_topic} (Cohérence via GENSIM) =====")

final_analysis = {} # Stockage des stats calculées

# --- Fonctions utilitaires pour l'analyse ---
# (La fonction analyze_metric reste la même)
def analyze_metric(scores, metric_name):
    """Calcule et retourne moyenne et écart-type pour une liste de scores."""
    valid_scores = [s for s in scores if s is not None and not np.isnan(s)]
    num_valid = len(valid_scores)
    if num_valid > 0:
        mean_score = np.mean(valid_scores)
        std_score = np.std(valid_scores) if num_valid >= 2 else 0.0
        return mean_score, std_score, num_valid
    else:
        return np.nan, np.nan, 0

# --- Analyse Custom ---
valid_custom_h = [h for h in robustness_results['custom']['h_matrices'] if h is not None]
print(f"\n--- Résultats Robustesse Custom (K={num_topic}, h={fixed_h_power_custom}, w={fixed_w_power_custom}) ---")

# 1. Erreur Reconstruction (Identique)
mean_err_custom, std_err_custom, num_success_err_custom = analyze_metric(
    robustness_results['custom']['errors'], 'Erreur Reconstruction'
)
final_analysis['custom_error_mean'] = mean_err_custom
final_analysis['custom_error_std'] = std_err_custom
if num_success_err_custom > 0:
    print(f"  Erreur Reconstruction: Moy={mean_err_custom:.4f}, Etd={std_err_custom:.4f} (sur {num_success_err_custom}/{N_ROBUSTNESS_RUNS} runs NMF valides)")
else:
    print("  Erreur Reconstruction: Aucun run NMF réussi.")

# 2. Stabilité Topics (Similarité Cosinus) (Identique)
num_success_h_custom = len(valid_custom_h)
if num_success_h_custom >= 2:
    # Assurez-vous que calculate_topic_stability est définie et fonctionne comme attendu
    stability_custom = calculate_topic_stability(valid_custom_h)
    print(f"  Stabilité Topics (Sim Cos Moy): {stability_custom:.4f} (basé sur {num_success_h_custom} matrices H)")
    final_analysis['custom_topic_stability'] = stability_custom
elif num_success_h_custom == 1:
     print(f"  Stabilité Topics: 1 run NMF réussi, stabilité = 1.0")
     final_analysis['custom_topic_stability'] = 1.0
else:
    print(f"  Stabilité Topics: Pas assez de matrices H valides ({num_success_h_custom}) pour calculer.")
    final_analysis['custom_topic_stability'] = np.nan

# 3. Cohérence c_NPMI (via GENSIM)
if GENSIM_AVAILABLE:
    mean_coh_custom, std_coh_custom, num_success_coh_custom = analyze_metric(
        robustness_results['custom']['coherence_scores'], f'Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE})'
    )
    final_analysis['custom_coherence_mean'] = mean_coh_custom
    final_analysis['custom_coherence_std'] = std_coh_custom
    if num_success_coh_custom > 0:
        # Affichage des scores de cohérence
        print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Moy={mean_coh_custom:.4f}, Etd={std_coh_custom:.4f} (sur {num_success_coh_custom}/{N_ROBUSTNESS_RUNS} runs avec score valide)")
    else:
        print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Aucun score valide calculé (vérifier erreurs Gensim ci-dessus si applicable).")
else:
    print("  Cohérence c_NPMI (GENSIM): Calcul désactivé (import/dict échoué).")
    final_analysis['custom_coherence_mean'] = np.nan
    final_analysis['custom_coherence_std'] = np.nan


# --- Analyse NNDSVD ---
valid_nndsvd_h = [h for h in robustness_results['nndsvd']['h_matrices'] if h is not None]
print(f"\n--- Résultats Robustesse NNDSVD (K={num_topic}) ---")

# 1. Erreur Reconstruction (Identique)
mean_err_nndsvd, std_err_nndsvd, num_success_err_nndsvd = analyze_metric(
    robustness_results['nndsvd']['errors'], 'Erreur Reconstruction'
)
final_analysis['nndsvd_error_mean'] = mean_err_nndsvd
final_analysis['nndsvd_error_std'] = std_err_nndsvd
if num_success_err_nndsvd > 0:
    print(f"  Erreur Reconstruction: Moy={mean_err_nndsvd:.4f}, Etd={std_err_nndsvd:.4f} (sur {num_success_err_nndsvd}/{N_ROBUSTNESS_RUNS} runs NMF valides)")
else:
    print("  Erreur Reconstruction: Aucun run NMF réussi.")

# 2. Stabilité Topics (Similarité Cosinus) (Identique)
num_success_h_nndsvd = len(valid_nndsvd_h)
if num_success_h_nndsvd >= 2:
    stability_nndsvd = calculate_topic_stability(valid_nndsvd_h)
    print(f"  Stabilité Topics (Sim Cos Moy): {stability_nndsvd:.4f} (basé sur {num_success_h_nndsvd} matrices H)")
    final_analysis['nndsvd_topic_stability'] = stability_nndsvd
elif num_success_h_nndsvd == 1:
    print(f"  Stabilité Topics: 1 run NMF réussi, stabilité = 1.0")
    final_analysis['nndsvd_topic_stability'] = 1.0
else:
    print(f"  Stabilité Topics: Pas assez de matrices H valides ({num_success_h_nndsvd})")
    final_analysis['nndsvd_topic_stability'] = np.nan

# 3. Cohérence c_NPMI (via GENSIM)
if GENSIM_AVAILABLE:
    mean_coh_nndsvd, std_coh_nndsvd, num_success_coh_nndsvd = analyze_metric(
        robustness_results['nndsvd']['coherence_scores'], f'Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE})'
    )
    final_analysis['nndsvd_coherence_mean'] = mean_coh_nndsvd
    final_analysis['nndsvd_coherence_std'] = std_coh_nndsvd
    if num_success_coh_nndsvd > 0:
        # Affichage des scores de cohérence
        print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Moy={mean_coh_nndsvd:.4f}, Etd={std_coh_nndsvd:.4f} (sur {num_success_coh_nndsvd}/{N_ROBUSTNESS_RUNS} runs avec score valide)")
    else:
        print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Aucun score valide calculé.")
else:
    print("  Cohérence c_NPMI (GENSIM): Calcul désactivé.")
    final_analysis['nndsvd_coherence_mean'] = np.nan
    final_analysis['nndsvd_coherence_std'] = np.nan


# --- Comparaison finale de robustesse ---
# (La logique de comparaison reste la même, mais utilise maintenant les valeurs calculées avec GENSIM)
print(f"\n--- Comparaison Robustesse K={num_topic} (Cohérence via GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}) ---")

custom_valid_error = not np.isnan(final_analysis.get('custom_error_mean', np.nan))
nndsvd_valid_error = not np.isnan(final_analysis.get('nndsvd_error_mean', np.nan))
custom_valid_topic = not np.isnan(final_analysis.get('custom_topic_stability', np.nan))
nndsvd_valid_topic = not np.isnan(final_analysis.get('nndsvd_topic_stability', np.nan))
custom_valid_coh = not np.isnan(final_analysis.get('custom_coherence_mean', np.nan))
nndsvd_valid_coh = not np.isnan(final_analysis.get('nndsvd_coherence_mean', np.nan))

# Comparaison Erreur Reconstruction
if custom_valid_error and nndsvd_valid_error:
     print("  Erreur Reconstruction:")
     if final_analysis['custom_error_mean'] < final_analysis['nndsvd_error_mean']: print("    -> Moyenne plus basse pour Custom")
     else: print("    -> Moyenne plus basse pour NNDSVD (ou égale)")
     if final_analysis['custom_error_std'] < final_analysis['nndsvd_error_std']: print("    -> Plus stable (Etd plus bas) pour Custom")
     else: print("    -> Plus stable (Etd plus bas) pour NNDSVD (ou égale)")
elif custom_valid_error: print("  Erreur Reconstruction: Seul Custom a des résultats valides.")
elif nndsvd_valid_error: print("  Erreur Reconstruction: Seul NNDSVD a des résultats valides.")
else: print("  Erreur Reconstruction: Non évaluable.")

# Comparaison Stabilité Topics (Similarité Cosinus)
if custom_valid_topic and nndsvd_valid_topic:
    print("  Stabilité Topics (Similarité Cos):")
    if final_analysis['custom_topic_stability'] > final_analysis['nndsvd_topic_stability']: print("    -> Plus stable (Sim Cos Moy plus haute) pour Custom")
    else: print("    -> Plus stable (Sim Cos Moy plus haute) pour NNDSVD (ou égale)")
elif custom_valid_topic: print("  Stabilité Topics: Seul Custom a des résultats valides.")
elif nndsvd_valid_topic: print("  Stabilité Topics: Seul NNDSVD a des résultats valides.")
else: print("  Stabilité Topics: Non évaluable.")

# Comparaison Cohérence c_NPMI (GENSIM)
if custom_valid_coh and nndsvd_valid_coh:
     print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}):")
     if final_analysis['custom_coherence_mean'] > final_analysis['nndsvd_coherence_mean']: print("    -> Moyenne plus haute pour Custom")
     else: print("    -> Moyenne plus haute pour NNDSVD (ou égale)")
     if final_analysis['custom_coherence_std'] < final_analysis['nndsvd_coherence_std']: print("    -> Plus stable (Etd plus bas) pour Custom")
     else: print("    -> Plus stable (Etd plus bas) pour NNDSVD (ou égale)")
elif custom_valid_coh: print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Seul Custom a des résultats valides.")
elif nndsvd_valid_coh: print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Seul NNDSVD a des résultats valides.")
elif not GENSIM_AVAILABLE: print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Calcul désactivé (import/dict échoué).")
else: print(f"  Cohérence c_NPMI (GENSIM, w={WINDOW_SIZE_FOR_COHERENCE}): Non évaluable (aucun run valide).")


print(f"\n===== FIN TEST DE ROBUSTESSE ÉTENDU pour K={num_topic} (Cohérence via GENSIM) =====")

# FIN DU SCRIPT PRINCIPAL (GENSIM)

Bibliothèque GENSIM pour la cohérence (CoherenceModel, Dictionary) trouvée.
Début du test de robustesse ÉTENDU (avec c_NPMI via GENSIM) pour K=15 uniquement.
Heure: 10:53:13 UTC, Lieu: Toulouse

Utilisation du scénario custom fixe : K=15, h_power=1.0, w_power=0.7
Nombre de runs de robustesse par méthode : 20
Graine solveur fixe pour custom : 1
Graines utilisées pour l'initialisation : 0 à 19
Paramètres c_NPMI (GENSIM) : topk=20, window_size=30

Vérification des prérequis...
Prérequis vérifiés.
Création du dictionnaire Gensim à partir de tokenized_documents...
Dictionnaire Gensim créé avec 61062 tokens uniques.

--- Test Robustesse: Scénario Custom Fixe (h=1.0, w=0.7) ---


  Robustesse Custom (K=15):   0%|          | 0/20 [00:00<?, ?seed/s]


--- Test Robustesse: NNDSVD ---


  Robustesse NNDSVD (K=15):   0%|          | 0/20 [00:00<?, ?seed/s]


===== ANALYSE DE ROBUSTESSE FINALE POUR K=15 (Cohérence via GENSIM) =====

--- Résultats Robustesse Custom (K=15, h=1.0, w=0.7) ---
  Erreur Reconstruction: Moy=8602.8513, Etd=0.6599 (sur 20/20 runs NMF valides)
  Stabilité Topics (Sim Cos Moy): 0.8950 (basé sur 20 matrices H)
  Cohérence c_NPMI (GENSIM, w=30): Moy=0.0887, Etd=0.0033 (sur 20/20 runs avec score valide)

--- Résultats Robustesse NNDSVD (K=15) ---
  Erreur Reconstruction: Moy=8602.2552, Etd=0.3247 (sur 20/20 runs NMF valides)
  Stabilité Topics (Sim Cos Moy): 0.8869 (basé sur 20 matrices H)
  Cohérence c_NPMI (GENSIM, w=30): Moy=0.0903, Etd=0.0017 (sur 20/20 runs avec score valide)

--- Comparaison Robustesse K=15 (Cohérence via GENSIM, w=30) ---
  Erreur Reconstruction:
    -> Moyenne plus basse pour NNDSVD (ou égale)
    -> Plus stable (Etd plus bas) pour NNDSVD (ou égale)
  Stabilité Topics (Similarité Cos):
    -> Plus stable (Sim Cos Moy plus haute) pour Custom
  Cohérence c_NPMI (GENSIM, w=30):
    -> Moyenne plus 

In [60]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 649.1/649.1 kB 10.4 MB/s eta 0:00:00


In [25]:
def extract_top_words(H, feature_names, n_top_words):
    """Extrait les n_top_words de chaque topic à partir de la matrice H."""
    topics = []
    num_topics, num_features = H.shape
    if len(feature_names) != num_features:
        print(f"ERREUR extract_top_words: Nb features H ({num_features}) != Nb feature_names ({len(feature_names)})")
        return [] # Retourner vide pour indiquer l'échec

    for t in range(num_topics):
        # Indices triés par poids décroissant
        top_word_indices = np.argsort(H[t])[:-n_top_words - 1:-1]
        # Récupérer les mots correspondants
        words = [feature_names[i] for i in top_word_indices]
        topics.append(words)
    return topics # Retourne une list[list[str]]

In [26]:
!pip show gensim

Name: gensim
Version: 4.3.3
Summary: Python framework for fast Vector Space Modelling
Home-page: https://radimrehurek.com/gensim/
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.12/site-packages
Requires: numpy, scipy, smart-open
Required-by: octis


In [27]:
from gensim.models.coherencemodel import CoherenceModel

In [28]:
from gensim.corpora import Dictionary

In [31]:
tokenized_documents

NameError: name 'tokenized_documents' is not defined

In [33]:
tokenized_documents = tokenized_corpus

In [34]:



def calculate_coherence_gensim(topics, tokenized_corpus, gensim_dictionary, window_size, measure='c_npmi'):
    """Calcule le score de cohérence en utilisant Gensim CoherenceModel."""
    if not GENSIM_AVAILABLE or gensim_dictionary is None: # Vérifier si Gensim est dispo et si le dictionnaire a été créé
        print("Skipping coherence calculation: Gensim not available or dictionary creation failed.")
        return np.nan

    # Vérifications basiques
    if not topics:
        print("WARN calculate_coherence_gensim: La liste de topics (top words) est vide.")
        return np.nan
    if not tokenized_corpus:
         print("WARN calculate_coherence_gensim: Le corpus tokenisé est vide.")
         return np.nan
    if not isinstance(topics, list) or not all(isinstance(topic, list) for topic in topics):
         print("WARN calculate_coherence_gensim: `topics` doit être une liste de listes de mots.")
         return np.nan

    try:
        # Initialiser le modèle de cohérence Gensim
        cm = CoherenceModel(
            topics=topics,                 # La liste de listes de mots top (pré-extraits)
            texts=tokenized_corpus,        # Le corpus tokenisé complet (list[list[str]])
            dictionary=gensim_dictionary,  # Le dictionnaire Gensim créé précédemment
            coherence=measure,             # Type de cohérence ('c_npmi', 'c_v', etc.)
            window_size=window_size,       # Taille de la fenêtre glissante pour c_npmi
            topn=len(topics[0]) if topics else n_top_words # Nombre de mots top utilisés par la mesure interne (redondant si 'topics' est fourni, mais bonne pratique de le spécifier)
        )
        # Obtenir le score
        score = cm.get_coherence()
        return score
    except Exception as e:
        print(f"ERREUR pendant le calcul de la cohérence ({measure}) avec GENSIM: {e}")
        import traceback
        traceback.print_exc() # Pour plus de détails sur l'erreur Gensim
        return np.nan # Retourner NaN en cas d'erreur


In [36]:
import time
import numpy as np
import logging
from sklearn.decomposition import NMF
import optuna # Import Optuna

# --- Configuration du Logging (Format simplifié) ---
# Configurez le logging comme vous le souhaitez (ex: fichier, niveau)
# Format simplifié pour éviter les erreurs avec les logs externes
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    force=True) # Ajout de force=True

optuna.logging.set_verbosity(optuna.logging.WARNING)

# <<< MODIFICATION 2 : Réduire la verbosité de Gensim >>>
# Ne montrera que les warnings et erreurs de Gensim
logging.getLogger('gensim').setLevel(logging.WARNING)

logger = logging.getLogger()

# --- Prérequis supposés définis ailleurs ---
# Assurez-vous que ces variables existent et sont correctement initialisées AVANT ce bloc
# tfidf = ... (votre matrice TF-IDF sparse ou dense)
# tokenized_documents = ... (liste de listes de tokens, ex: [['mot1', 'mot2'], ['mot3']])
# tfidf_feature_names = ... (liste des noms de features/mots du TF-IDF)
# epsilon = ... (petite valeur, ex: 1e-6)
# create_decreasing_H = ... (votre fonction)
# create_decreasing_W = ... (votre fonction)
# extract_top_words = ... (votre fonction)
# calculate_coherence_gensim = ... (votre fonction, assurez-vous qu'elle accepte 'texts')

# --- Dimensions des données ---
try:
    n_samples, n_features = tfidf.shape
except NameError:
    logging.error("La variable 'tfidf' n'est pas définie. Veuillez la charger ou la créer.")
    # Arrêter ou définir des valeurs par défaut si nécessaire
    exit() # Ou gérer l'erreur autrement

# --- Constantes et Prérequis (Définis EN DEHORS de l'objectif) ---
RANDOM_SEED = 1
INIT_RANDOM_SEED = 42
NMF_SOLVER = 'cd'
NMF_INIT_METHOD = 'custom'
MAX_ITER = 10000          # Exemple: nombre max d'itérations pour NMF
ALPHA_W = 0.0           # Exemple: régularisation L1/L2 pour W
ALPHA_H = 0.0           # Exemple: régularisation L1/L2 pour H ('auto' souvent géré par solver 'cd')
L1_RATIO = 0.0          # Exemple: ratio L1 (0=L2, 1=L1)
N_TOP_WORDS = 10        # Exemple: nombre de mots clés à extraire par topic
epsilon = 1e-6        # Déplacé vers les prérequis supposés
SCENARIO_NAME = "NMF_Optuna_Optim" # Nom pour les logs

# --- Vérification et Initialisation Gensim ---
GENSIM_AVAILABLE = True

COHERENCE_WINDOW_SIZE = 30 # Taille de fenêtre pour C_NPMI si Gensim utilisé

# ==============================================================
# FONCTION OBJECTIVE POUR OPTUNA (MODIFIÉE)
# ==============================================================
def objective(trial):
    """
    Fonction exécutée par Optuna pour chaque essai.
    Teste une combinaison d'hyperparamètres et retourne le score de cohérence.
    Retourne -float('inf') en cas d'échec pour permettre la maximisation.
    """
    trial_num = trial.number # Récupérer le numéro de l'essai pour les logs

    # 1. Suggérer les hyperparamètres à tester pour cet essai
    num_topic = trial.suggest_int('num_topic', 5, 70) # K : Nombre de topics
    h_num_items_decay = trial.suggest_int('h_num_items_decay', 50, 500) # Pour H init
    h_power = trial.suggest_float('h_power', 0.5, 10.0)           # Pour H init
    h_noise_level = trial.suggest_float('h_noise_level', 1e-4, 0.5, log=True) # Pour H init
    w_power = trial.suggest_float('w_power', 0.5, 10.0)           # Pour W init
    w_noise_level = trial.suggest_float('w_noise_level', 1e-4, 0.5, log=True) # Pour W init
    initial_peak = trial.suggest_float('initial_peak', 0.5, 10.0) # Pour W init

    logging.info(f"Trial {trial_num}: Début essai avec K={num_topic}, h_decay={h_num_items_decay}, h_pow={h_power:.2f}, h_noise={h_noise_level:.4f}, w_pow={w_power:.2f}, w_noise={w_noise_level:.4f}, intial_peak={initial_peak:.2f}")

    # ==============================================================
    # 1. Créer les matrices d'initialisation personnalisées H et W
    # ==============================================================
    current_H_init, current_W_init = None, None
    expected_H_shape = (num_topic, n_features)
    expected_W_shape = (n_samples, num_topic)

   # logging.info(f"Trial {trial_num}: Tentative de création des matrices d'initialisation H et W pour K={num_topic}...")
    try:
        # Assurez-vous que les fonctions et 'epsilon' sont disponibles
        current_H_init = create_decreasing_H(
            num_topic, n_features, h_num_items_decay, h_power, h_noise_level, INIT_RANDOM_SEED, epsilon, initial_peak
        )
        current_W_init = create_decreasing_W(
            n_samples, num_topic, w_power, w_noise_level, INIT_RANDOM_SEED, epsilon, initial_peak
        )

        # Vérification rigoureuse
        if current_W_init is None or current_H_init is None:
            raise ValueError("La création d'une matrice d'initialisation (H ou W) a échoué (résultat None).")
        if current_W_init.shape != expected_W_shape or current_H_init.shape != expected_H_shape:
            raise ValueError(
                f"Dimensions initialisation INCOHÉRENTES: "
                f"W={current_W_init.shape} (attendu {expected_W_shape}), "
                f"H={current_H_init.shape} (attendu {expected_H_shape})"
            )

   #     logging.info(f"Trial {trial_num}: Initialisation personnalisée (H/W) créée avec succès pour K={num_topic}.")
        init_creation_success = True

    except NameError as e_name:
         logging.error(
            f"Trial {trial_num}: ERREUR - Variable ou fonction MANQUANTE lors de la création de l'initialisation (K={num_topic}): {e_name}. Assurez-vous que 'epsilon', 'create_decreasing_H', 'create_decreasing_W' sont définies.",
            exc_info=False
        )
         init_creation_success = False
    except Exception as e_init:
        logging.error(
            f"Trial {trial_num}: ERREUR lors de la création ou validation de l'initialisation (K={num_topic}): {e_init}",
            exc_info=False
        )
        init_creation_success = False

    # Si l'initialisation échoue, cet essai est invalide.
    if not init_creation_success:
        logging.warning(f"Trial {trial_num}: Échec de l'initialisation, retour de -inf.")
        return -float('inf') # Retourner une mauvaise valeur pour la maximisation

    # ==============================================================
    # 2. Sauvegarder les matrices d'initialisation (Optionnel)
    # ==============================================================
    # Décommenter et adapter si nécessaire (définir init_csv_dir)
    # init_csv_dir = "/chemin/optionnel/pour/sauvegarde"
    # if init_creation_success and 'init_csv_dir' in locals() and init_csv_dir:
    #     try:
    #         # ... Code pour sauvegarder H et W ...
    #         logging.info(f"Trial {trial_num}: Matrices d'initialisation K={num_topic} sauvegardées.")
    #     except Exception as e_save:
    #         logging.error(f"Trial {trial_num}: Échec sauvegarde matrices K={num_topic}: {e_save}", exc_info=False)

    # ==============================================================
    # 3. Exécuter NMF et calculer la cohérence
    # ==============================================================
    nmf_model = None
  #  reconstruction_error = float('inf')
    coherence_score = -float('inf') # Initialiser à -inf pour maximisation
    #duration = 0.0
    nmf_fit_success = False
    topic_words_custom = []

   # logging.info(f"Trial {trial_num}: Initialisation NMF (K={num_topic}) avec méthode '{NMF_INIT_METHOD}'...")
    try:
        nmf_model = NMF(
            n_components=num_topic,
            init=NMF_INIT_METHOD,
            solver=NMF_SOLVER,
            random_state=RANDOM_SEED + trial_num, # Seed différent par essai
            max_iter=MAX_ITER,
            alpha_W=ALPHA_W,
            alpha_H=ALPHA_H,
            l1_ratio=L1_RATIO,
            # tol=1e-4 # Vous pouvez aussi optimiser la tolérance
        )
    except Exception as e_nmf_init:
         logging.error(f"Trial {trial_num}: ERREUR lors de l'initialisation de sklearn.NMF (K={num_topic}): {e_nmf_init}", exc_info=False)
         logging.warning(f"Trial {trial_num}: Échec init NMF, retour de -inf.")
         return -float('inf')


    #start_time = time.time()
   # logging.info(f"Trial {trial_num}: Démarrage NMF.fit (K={num_topic})...")
    try:
        # Utiliser .copy() car NMF modifie W et H en place avec init='custom'
        W_fit = current_W_init.copy()
        H_fit = current_H_init.copy()
        # Assurez-vous que 'tfidf' est disponible
        nmf_model.fit(tfidf, W=W_fit, H=H_fit)

       # reconstruction_error = nmf_model.reconstruction_err_
        nmf_fit_success = True
       # logging.info(f"Trial {trial_num}: NMF.fit (K={num_topic}) terminé. Erreur={reconstruction_error:.4f}")

    except NameError as e_name:
        logging.error(f"Trial {trial_num}: ERREUR - Variable 'tfidf' MANQUANTE durant NMF.fit (K={num_topic}): {e_name}", exc_info=False)
        nmf_fit_success = False
    except Exception as e_fit:
        logging.error(f"Trial {trial_num}: ERREUR durant NMF.fit (K={num_topic}): {e_fit}", exc_info=False)
        nmf_fit_success = False # Marquer comme échoué

  #  finally:
      #  duration = time.time() - start_time
      #  logging.info(f"Trial {trial_num}: Temps écoulé pour NMF.fit (K={num_topic}): {duration:.2f}s")

    # Si NMF.fit a échoué, retourner une mauvaise valeur
    if not nmf_fit_success:
        logging.warning(f"Trial {trial_num}: Échec NMF.fit, retour de -inf.")
        return -float('inf')

    # --- Calcul de la cohérence et extraction des mots SEULEMENT SI NMF.fit a réussi ---
   # logging.info(f"Trial {trial_num}: Extraction mots clés et calcul cohérence (K={num_topic})...")
    try:
        # 1. Extraire les mots clés
        # Assurez-vous que 'tfidf_feature_names' et 'extract_top_words' sont disponibles
        topic_words_custom = extract_top_words(nmf_model.components_, tfidf_feature_names, N_TOP_WORDS)

        if not topic_words_custom:
            logging.warning(f"Trial {trial_num}: Aucun mot clé extrait pour K={num_topic}. Cohérence non calculée.")
            coherence_score = -float('inf') # Pas de cohérence si pas de mots

        # 2. Calculer la cohérence si possible
        elif GENSIM_AVAILABLE and gensim_dictionary:
           # logging.info(f"Trial {trial_num}: Calcul cohérence C_NPMI (Gensim) pour K={num_topic}...")
            try:
                # Assurez-vous que 'tokenized_documents', 'gensim_dictionary',
                # et 'calculate_coherence_gensim' sont disponibles et que
                # calculate_coherence_gensim accepte bien le paramètre 'texts'
                coherence_score_calc = calculate_coherence_gensim(
                    topics=topic_words_custom,
                    tokenized_corpus=tokenized_documents,    # Passez les documents tokenisés ici
                    gensim_dictionary=gensim_dictionary,
                    window_size=COHERENCE_WINDOW_SIZE
                    # coherence_type='c_npmi' # si votre fonction le gère
                )
                # Vérifier si le calcul a retourné NaN
                if np.isnan(coherence_score_calc):
                     logging.warning(f"Trial {trial_num}: Calcul cohérence a retourné NaN pour K={num_topic}. Retour de -inf.")
                     coherence_score = -float('inf')
                else:
                     coherence_score = coherence_score_calc # Assigner le score calculé
                  #   logging.info(f"Trial {trial_num}: Score cohérence C_NPMI (Gensim) pour K={num_topic}: {coherence_score:.4f}")

            except TypeError as e_coh_type:
                 # Erreur spécifique si 'texts' n'est pas accepté
                 if 'unexpected keyword argument \'texts\'' in str(e_coh_type):
                      logging.error(f"Trial {trial_num}: ERREUR - Votre fonction 'calculate_coherence_gensim' n'accepte pas l'argument 'texts'. Veuillez corriger sa définition. {e_coh_type}", exc_info=False)
                 else:
                      logging.error(f"Trial {trial_num}: ERREUR de type durant calcul cohérence Gensim K={num_topic}: {e_coh_type}", exc_info=False)
                 coherence_score = -float('inf') # Marquer l'échec
            except NameError as e_name:
                 logging.error(f"Trial {trial_num}: ERREUR - Variable MANQUANTE durant calcul cohérence ({e_name}). Vérifiez 'tokenized_documents', 'gensim_dictionary', 'calculate_coherence_gensim'.", exc_info=False)
                 coherence_score = -float('inf') # Marquer l'échec
            except Exception as e_coh:
                logging.error(f"Trial {trial_num}: ERREUR durant calcul cohérence Gensim K={num_topic}: {e_coh}", exc_info=False)
                coherence_score = -float('inf') # Marquer l'échec

        else:
            if not GENSIM_AVAILABLE:
                 logging.warning(f"Trial {trial_num}: Gensim non disponible. Score cohérence non calculé pour K={num_topic}. Retour de -inf.")
            elif not gensim_dictionary:
                 logging.warning(f"Trial {trial_num}: Dictionnaire Gensim non créé. Score cohérence non calculé pour K={num_topic}. Retour de -inf.")
            coherence_score = -float('inf') # Pas de calcul possible

    except NameError as e_name:
        logging.error(f"Trial {trial_num}: ERREUR - Variable MANQUANTE extraction mots clés K={num_topic} ({e_name}). Vérifiez 'tfidf_feature_names', 'extract_top_words'.", exc_info=False)
        coherence_score = -float('inf')
    except Exception as e_extract:
        logging.error(f"Trial {trial_num}: ERREUR extraction mots clés K={num_topic}: {e_extract}", exc_info=False)
        coherence_score = -float('inf') # Pas de cohérence si extraction échoue

    # --- Retourner la métrique à optimiser ---
    # coherence_score est soit la valeur calculée, soit -inf si une étape a échoué
    logging.info(f"Trial {trial_num}: Fin essai. Score retourné: {coherence_score}")
    # Gérer explicitement le cas où le score est -inf (même si Optuna le gère)
    if coherence_score == -float('inf'):
         # Vous pourriez lever optuna.TrialPruned() ici si vous voulez que le pruner agisse
         # raise optuna.TrialPruned("Échec critique dans le calcul de l'objectif.")
         pass # Ou juste retourner -inf

    # Vérifier une dernière fois si NaN (ne devrait pas arriver avec la logique ci-dessus, mais par sécurité)
    if np.isnan(coherence_score):
        logging.error(f"Trial {trial_num}: Score final est NaN ! Retour de -inf.")
        return -float('inf')

    return coherence_score

# ==============================================================
# OPTIMISATION AVEC OPTUNA
# ==============================================================

# Vérifier que les prérequis essentiels existent
if 'tfidf' not in globals() or 'tokenized_documents' not in globals() or 'tfidf_feature_names' not in globals():
        logging.critical("Variables essentielles (tfidf, tokenized_documents, tfidf_feature_names) non définies. Arrêt.")
        # exit()

# Crée une étude Optuna.
study = None
try:
    study = optuna.create_study(direction='maximize',
                                study_name=SCENARIO_NAME,
                                # Utiliser un pruner peut accélérer si certains essais sont très lents et peu prometteurs
                                pruner=optuna.pruners.MedianPruner(n_warmup_steps=5, n_min_trials=5)) # Prune après 5 essais complets
except Exception as e_study:
    logging.critical(f"Impossible de créer l'étude Optuna : {e_study}", exc_info=True)
    # exit()


best_trial = None
if study:
    # Nombre d'essais à réaliser
    n_trials = 200 # Adaptez ce nombre
    timeout_seconds = 3600 # Adaptez ou mettez None

    logging.info(f"=== Démarrage de l'optimisation Optuna pour '{study.study_name}' avec {n_trials} essais (timeout={timeout_seconds}s) ===")

    try:
        # Lance l'optimisation
        study.optimize(objective, n_trials=n_trials, timeout=timeout_seconds)
    except KeyboardInterrupt:
        logging.warning("Optimisation interrompue par l'utilisateur (KeyboardInterrupt).")
    except NameError as e_opt_name:
        logging.error(f"Une variable MANQUANTE a été détectée pendant study.optimize: {e_opt_name}. Vérifiez les dépendances globales.", exc_info=True)
    except Exception as e_opt:
        logging.error(f"Une erreur majeure est survenue durant l'optimisation: {e_opt}", exc_info=True)

    # --- Affichage des résultats ---
    logging.info("=== Optimisation terminée ===")

    # Vérifier s'il y a un meilleur essai (pourrait ne pas exister si tout a échoué ou a été interrompu tôt)
    try:
        best_trial = study.best_trial
        logging.info(f"Meilleur essai trouvé : Numéro {best_trial.number}")

        best_value = best_trial.value
        # Vérifier si la meilleure valeur est valide (pas -inf ou None)
        if best_value is not None and best_value != -float('inf'):
            logging.info(f"  Meilleur score (cohérence) : {best_value:.5f}")
        elif best_value == -float('inf'):
            logging.warning("  Tous les essais réussis ont retourné -inf (échec interne probable).")
        else:
            logging.warning("  Aucun essai n'a produit un score de cohérence valide ou n'a pu être évalué.")

        logging.info("  Meilleurs hyperparamètres :")
        for key, value in best_trial.params.items():
            logging.info(f"    {key}: {value}")

    except ValueError:
        logging.warning("Aucun essai terminé avec succès n'a été trouvé dans l'étude (study.best_trial a échoué).")
    except Exception as e_results:
        logging.error(f"Erreur lors de l'affichage des meilleurs résultats : {e_results}")


    # Optionnel : Visualisations Optuna
    try:
        # Vérifier si des essais ont été complétés avant de tenter de générer les graphiques
        completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
        if completed_trials:
            logging.info("Génération des visualisations Optuna...")
            # S'assurer que plotly est installé
         #   import plotly # Mettre dans un try/except si optionnel

            # Importance des hyperparamètres
            fig1 = optuna.visualization.plot_param_importances(study)
            fig1.show()
            # fig1.write_html("param_importances.html")

            # Historique d'optimisation
            fig2 = optuna.visualization.plot_optimization_history(study)
            fig2.show()
            # fig2.write_html("optimization_history.html")

            # Slice plot (choisir quelques paramètres pertinents)
            # Vérifier quels paramètres sont effectivement dans l'étude
            params_to_plot = [p for p in ['num_topic', 'h_noise_level', 'w_noise_level', 'h_power'] if p in study.best_params]
            if params_to_plot:
                fig3 = optuna.visualization.plot_slice(study, params=params_to_plot)
                fig3.show()
                # fig3.write_html("slice_plot.html")
            else:
                logging.info("Aucun paramètre pertinent trouvé pour le slice plot.")

        else:
            logging.warning("Aucun essai complété, impossible de générer les visualisations Optuna.")

    except ImportError:
        logging.warning("Bibliothèques de visualisation (plotly, matplotlib) non trouvées. Saut des graphiques.")
    except Exception as e_vis:
        logging.warning(f"Impossible de générer les visualisations Optuna : {e_vis}", exc_info=False) # Mettre True pour debug

2025-04-12 11:01:05,888 - INFO - === Démarrage de l'optimisation Optuna pour 'NMF_Optuna_Optim' avec 200 essais (timeout=3600s) ===
2025-04-12 11:01:05,893 - INFO - Trial 0: Début essai avec K=40, h_decay=160, h_pow=9.43, h_noise=0.0005, w_pow=5.76, w_noise=0.0035, intial_peak=8.19
2025-04-12 11:06:32,134 - INFO - Trial 0: Fin essai. Score retourné: 0.17376070373743635
2025-04-12 11:06:32,144 - INFO - Trial 1: Début essai avec K=19, h_decay=400, h_pow=4.12, h_noise=0.0001, w_pow=7.76, w_noise=0.0015, intial_peak=4.65
2025-04-12 11:09:59,525 - INFO - Trial 1: Fin essai. Score retourné: 0.15560651880411336
2025-04-12 11:09:59,527 - INFO - Trial 2: Début essai avec K=60, h_decay=468, h_pow=6.66, h_noise=0.3498, w_pow=2.68, w_noise=0.0007, intial_peak=1.37
2025-04-12 11:12:10,506 - INFO - Trial 2: Fin essai. Score retourné: 0.18054932691624406
2025-04-12 11:12:10,512 - INFO - Trial 3: Début essai avec K=24, h_decay=181, h_pow=9.04, h_noise=0.0084, w_pow=1.93, w_noise=0.0002, intial_peak=8.

In [95]:
best_params

{'num_topic': 7,
 'h_num_items_decay': 169,
 'h_power': 4.985014922686625,
 'h_noise_level': 0.018616890492508073,
 'w_power': 4.452134245034107,
 'w_noise_level': 0.03383757092291506}

In [98]:
best_params = best_trial.params

import logging
import time
import numpy as np
# Assurez-vous que toutes les dépendances nécessaires sont importées :
# import optuna # Pas directement nécessaire ici, mais best_params vient de là
# from sklearn.decomposition import NMF
# from gensim.corpora import Dictionary # Si non déjà importé
# from gensim.models.coherencemodel import CoherenceModel # Si non déjà importé

# --- Configuration du Logging (Assurez-vous qu'il est configuré avant) ---
# Exemple:
# logging.basicConfig(level=logging.INFO,
#                     format='%(asctime)s - %(levelname)s - %(message)s',
#                     force=True)
# logger = logging.getLogger()

# --- Assurez-vous que ces variables sont définies AVANT ce bloc ---
# ----- Issues d'Optuna -----
# best_params: dict # Dictionnaire contenant les meilleurs hyperparamètres trouvés
# ----- Données & Pré-traitement -----
# tfidf: np.ndarray ou sparse matrix # Matrice TF-IDF des données
# n_samples: int # Nombre de documents
# n_features: int # Nombre de features (mots)
# tfidf_feature_names: list[str] # Liste des noms des features
# tokenized_documents: list[list[str]] # Corpus tokenisé (pour Gensim)
# ----- Paramètres NMF -----
# NMF_INIT_METHOD, NMF_SOLVER: str # Paramètres NMF
# MAX_ITER: int # Max iterations pour NMF
# ALPHA_W, ALPHA_H, L1_RATIO: float # Paramètres de régularisation NMF
# N_TOP_WORDS: int # Nombre de mots clés à afficher par topic
# epsilon: float # Petite valeur pour éviter les zéros
# RANDOM_SEED, INIT_RANDOM_SEED: int # Graines aléatoires
# ----- Fonctions Utilitaires -----
# create_decreasing_H, create_decreasing_W: functions # Fonctions d'initialisation
# extract_top_words: function # Fonction pour extraire les mots clés
# ----- Prérequis pour la Cohérence Gensim -----
# calculate_coherence_gensim: function # Fonction calculant la cohérence (acceptant 'texts')
# gensim_dictionary: gensim.corpora.Dictionary # Dictionnaire Gensim créé sur tokenized_documents
# GENSIM_AVAILABLE: bool # Drapeau indiquant si Gensim est disponible/utilisable
# COHERENCE_WINDOW_SIZE: int # Taille de fenêtre pour la cohérence
# --------------------------------------------------------------------

# Dictionnaires globaux (supposés définis en dehors ou au début du script/notebook)
# Ces dictionnaires seront remplis par ce bloc
all_nmf_H = {}
all_nmf_W = {}
nmf_models = {}

# Variable pour stocker le score de cohérence final (initialisée)
final_coherence_score = None

# <<< --- DÉBUT DU BLOC DE RÉ-ENTRAÎNEMENT FINAL ET CALCUL DE COHÉRENCE --- >>>
logging.info("=" * 60)
logging.info(" LANCEMENT DU RÉ-ENTRAÎNEMENT FINAL + COHÉRENCE ".center(60, "="))
logging.info(f"Utilisation des meilleurs paramètres trouvés et de graines fixes.")

# Définir les graines fixes pour le ré-entraînement
final_model_seed = RANDOM_SEED       # Graine pour le solveur sklearn NMF
final_init_seed = INIT_RANDOM_SEED   # Graine pour les fonctions create_decreasing_H/W

# Extraire les meilleurs hyperparamètres nécessaires
try:
    # Récupération de la clé (nombre de topics) et autres paramètres
    best_num_topic = best_params['num_topic']
    best_h_num_items_decay = best_params['h_num_items_decay']
    best_h_power = best_params['h_power']
    best_h_noise_level = best_params['h_noise_level']
    best_w_power = best_params['w_power']
    best_w_noise_level = best_params['w_noise_level']
    # S'assurer que 'initial_peak' est récupéré si utilisé par vos fonctions create_
    # (Il manquait dans votre snippet original, ajout au cas où)
    best_initial_peak = best_params.get('initial_peak', 1.0) # Utiliser une valeur par défaut si non trouvé

    logging.info(f"Meilleurs paramètres récupérés: K={best_num_topic}, h_decay={best_h_num_items_decay}, h_pow={best_h_power:.2f}, ...")

except KeyError as e_key:
    logging.error(f"Erreur: Hyperparamètre clé manquant dans best_params: {e_key}. Arrêt.")
    # Peut-être sortir ou lever une exception plus spécifique
    raise SystemExit(f"Paramètre manquant: {e_key}")
except NameError:
    logging.error("Erreur: La variable 'best_params' n'est pas définie. Avez-vous lancé Optuna avant ? Arrêt.")
    # Peut-être sortir ou lever une exception plus spécifique
    raise SystemExit("Variable 'best_params' non définie.")


# --- Vérification des prérequis critiques avant de continuer ---
critical_vars = ['tfidf', 'n_samples', 'n_features', 'tfidf_feature_names',
                 'tokenized_documents', 'create_decreasing_H', 'create_decreasing_W',
                 'extract_top_words', 'calculate_coherence_gensim', 'gensim_dictionary',
                 'epsilon', 'NMF_INIT_METHOD', 'NMF_SOLVER', 'MAX_ITER',
                 'ALPHA_W', 'ALPHA_H', 'L1_RATIO', 'N_TOP_WORDS', 'RANDOM_SEED',
                 'INIT_RANDOM_SEED', 'GENSIM_AVAILABLE', 'COHERENCE_WINDOW_SIZE']
missing_vars = [var for var in critical_vars if var not in globals()]
if missing_vars:
    logging.error(f"Erreur: Variables/Fonctions critiques MANQUANTES : {', '.join(missing_vars)}. Arrêt.")
    raise SystemExit(f"Variables/Fonctions manquantes : {', '.join(missing_vars)}")


# 1. Recréer les matrices d'initialisation H et W
final_H_init, final_W_init = None, None
expected_H_shape = (best_num_topic, n_features)
expected_W_shape = (n_samples, best_num_topic)
logging.info(f"-> 1. Recréation des matrices H/W pour K={best_num_topic} avec seed_init={final_init_seed}...")
try:
    final_H_init = create_decreasing_H(
        best_num_topic, n_features, best_h_num_items_decay,
        best_h_power, best_h_noise_level,
        final_init_seed, epsilon, best_initial_peak # Ajout de initial_peak
    )
    final_W_init = create_decreasing_W(
        n_samples, best_num_topic, best_w_power,
        best_w_noise_level, final_init_seed, epsilon, best_initial_peak # Ajout de initial_peak
    )

    if final_W_init is None or final_H_init is None:
        raise ValueError("Échec de la création des matrices d'init finales (résultat None).")
    if final_W_init.shape != expected_W_shape or final_H_init.shape != expected_H_shape:
        raise ValueError(f"Dimensions initialisation finales INCOHÉRENTES: W={final_W_init.shape} attendu {expected_W_shape}, H={final_H_init.shape} attendu {expected_H_shape}")
    logging.info("   Matrices d'initialisation finales H/W créées avec succès.")

except Exception as e_init_final:
    logging.error(f"   ERREUR lors de la création/validation des matrices d'initialisation finales : {e_init_final}", exc_info=True)
    raise # Arrêter le processus si l'initialisation échoue

# 2. Initialiser le modèle NMF final
logging.info(f"-> 2. Initialisation du modèle NMF final (K={best_num_topic}) avec random_state={final_model_seed}...")
final_nmf_model = None # Initialiser à None avant le try
try:
    # Utilisation de la graine fixe RANDOM_SEED pour le random_state de NMF
    final_nmf_model = NMF(
        n_components=best_num_topic,
        init=NMF_INIT_METHOD, # 'custom'
        solver=NMF_SOLVER,
        random_state=final_model_seed, # <--- GRAINE FIXE ICI
        max_iter=MAX_ITER,             # Vous pourriez augmenter max_iter ici si désiré
        alpha_W=ALPHA_W,
        alpha_H=ALPHA_H,
        l1_ratio=L1_RATIO,
        # tol=1e-4 # Vous pouvez définir une tolérance fixe ici aussi
    )
    logging.info("   Modèle NMF final initialisé.")
except Exception as e_nmf_final_init:
    logging.error(f"   ERREUR lors de l'initialisation du modèle NMF final: {e_nmf_final_init}", exc_info=True)
    raise # Arrêter le processus

# 3. Entraîner (fit) le modèle NMF final
logging.info("-> 3. Entraînement (fit) du modèle NMF final...")
start_final_fit = time.time()
W_final_fit = None # Initialiser à None avant le try/catch
H_final_fit = None # Initialiser à None avant le try/catch
training_successful = False
try:
    # Important : utiliser .copy() car NMF(init='custom') modifie W et H en place
    W_final_fit = final_W_init.copy()
    H_final_fit = final_H_init.copy()

    # Entraînement du modèle. W_final_fit et H_final_fit seront modifiés en place.
    final_nmf_model.fit(tfidf, W=W_final_fit, H=H_final_fit)

    duration_final_fit = time.time() - start_final_fit
    logging.info(f"   Modèle NMF final entraîné avec succès en {duration_final_fit:.2f}s.")
    logging.info(f"   Erreur de reconstruction finale : {final_nmf_model.reconstruction_err_:.4f}")
    training_successful = True

    # Stockage des résultats si le fit a réussi
    logging.info(f"-> 4. Stockage des résultats pour K={best_num_topic}...")
    all_nmf_H[best_num_topic] = final_nmf_model.components_
    all_nmf_W[best_num_topic] = W_final_fit # W a été modifié en place
    nmf_models[best_num_topic] = final_nmf_model
    logging.info(f"   Résultats (modèle, W, H) pour K={best_num_topic} stockés avec succès.")

except Exception as e_fit_final:
    logging.error(f"   ERREUR durant l'entraînement (fit) du modèle NMF final: {e_fit_final}", exc_info=True)
    # Pas de stockage si le fit échoue

# 5. Afficher les mots clés et calculer la cohérence (SEULEMENT si l'entraînement a réussi)
if training_successful and best_num_topic in nmf_models:
    logging.info(f"-> 5. Extraction des mots clés et calcul de la cohérence C_NPMI...")
    try:
        # Récupérer le modèle et la matrice H depuis les dictionnaires/variables
        stored_model = nmf_models[best_num_topic]
        stored_H = all_nmf_H[best_num_topic] # Ou utiliser stored_model.components_

        # Extraction des mots clés
        logging.info(f"   Extraction des {N_TOP_WORDS} mots clés principaux...")
        final_topic_words = extract_top_words(stored_H, tfidf_feature_names, N_TOP_WORDS)

        if not final_topic_words:
             logging.warning("   Aucun mot clé n'a été extrait pour le modèle final. Impossible d'afficher ou de calculer la cohérence.")
        else:
            # Affichage des mots clés
            logging.info(f"   Mots clés principaux du modèle NMF final (K={best_num_topic}) :")
            for i, words in enumerate(final_topic_words):
                logging.info(f"     Topic {i}: {', '.join(words)}") # Affichage plus lisible

            # Calcul de la cohérence C_NPMI sur le modèle final
            logging.info(f"   Calcul de la cohérence C_NPMI (Gensim) pour le modèle final K={best_num_topic}...")
            if GENSIM_AVAILABLE and gensim_dictionary:
                try:
                    # Appel de la fonction de calcul de cohérence
                    calculated_score = calculate_coherence_gensim(
                        topics=final_topic_words,
                        tokenized_corpus=tokenized_documents,    # Corpus tokenisé
                        gensim_dictionary=gensim_dictionary, # Dictionnaire Gensim
                        window_size=COHERENCE_WINDOW_SIZE    # Taille de fenêtre
                        # coherence_type='c_npmi' # si votre fonction le gère
                    )

                    # Vérification du résultat
                    if np.isnan(calculated_score):
                        logging.warning("   Le calcul de cohérence a retourné NaN.")
                        final_coherence_score = np.nan # Ou None
                    else:
                        final_coherence_score = calculated_score
                        logging.info(f"   SCORE DE COHÉRENCE FINAL (C_NPMI) : {final_coherence_score:.5f}")

                except TypeError as e_coh_type:
                     # Gérer spécifiquement l'erreur si 'texts'/'tokenized_corpus' n'est pas le bon nom d'argument
                     logging.error(f"   ERREUR de type durant calcul cohérence final : {e_coh_type}. Vérifiez les arguments de 'calculate_coherence_gensim'.", exc_info=False)
                except NameError as e_name:
                     logging.error(f"   ERREUR - Variable MANQUANTE durant calcul cohérence final ({e_name}).", exc_info=False)
                except Exception as e_coh:
                    logging.error(f"   ERREUR durant calcul cohérence final : {e_coh}", exc_info=False)
            else:
                if not GENSIM_AVAILABLE:
                    logging.warning("   Gensim non disponible. Score de cohérence final non calculé.")
                elif not gensim_dictionary:
                    logging.warning("   Dictionnaire Gensim non disponible/créé. Score de cohérence final non calculé.")

    except NameError as e_name_final:
        logging.warning(f"   Attention : Variable MANQUANTE lors de l'extraction/calcul final pour K={best_num_topic}: {e_name_final}")
    except Exception as e_final_process:
        logging.warning(f"   Attention : Erreur lors de l'extraction/affichage/calcul final pour K={best_num_topic}: {e_final_process}")
else:
    if not training_successful:
        logging.warning("Le modèle final n'a pas été entraîné avec succès. Impossible d'extraire les mots clés ou de calculer la cohérence.")
    else: # Implique que best_num_topic n'est pas dans nmf_models, ce qui ne devrait pas arriver si training_successful est True
         logging.error("Incohérence : Entraînement marqué comme réussi, mais modèle non trouvé dans le dictionnaire.")

logging.info("=" * 60)
logging.info(" RÉ-ENTRAÎNEMENT ET CALCUL COHÉRENCE TERMINÉS ".center(60, "="))
logging.info("=" * 60)

# Affichage final du score (s'il a été calculé)
if final_coherence_score is not None and not np.isnan(final_coherence_score):
    logging.info(f"Score de cohérence C_NPMI final obtenu pour K={best_num_topic} : {final_coherence_score:.5f}")
elif np.isnan(final_coherence_score):
    logging.warning("Le calcul de cohérence final a résulté en NaN.")
else:
    logging.warning("Le score de cohérence final n'a pas pu être calculé (voir logs précédents pour la raison).")

# Optionnel: Vérifier que les dictionnaires ont été remplis (si training_successful)
if training_successful:
    logging.debug(f"Clés dans nmf_models: {list(nmf_models.keys())}")
    logging.debug(f"Clés dans all_nmf_W: {list(all_nmf_W.keys())}")
    logging.debug(f"Clés dans all_nmf_H: {list(all_nmf_H.keys())}")
    if best_num_topic in all_nmf_H:
         logging.debug(f"Shape H pour K={best_num_topic}: {all_nmf_H[best_num_topic].shape}")
    if best_num_topic in all_nmf_W:
         logging.debug(f"Shape W pour K={best_num_topic}: {all_nmf_W[best_num_topic].shape}")
# <<< --- FIN DU BLOC --- >>>

2025-04-11 15:05:07,342 - INFO - ============================================================
2025-04-11 15:05:07,343 - INFO - ====== LANCEMENT DU RÉ-ENTRAÎNEMENT FINAL + COHÉRENCE ======
2025-04-11 15:05:07,344 - INFO - Utilisation des meilleurs paramètres trouvés et de graines fixes.
2025-04-11 15:05:07,345 - INFO - Meilleurs paramètres récupérés: K=5, h_decay=339, h_pow=6.45, ...
2025-04-11 15:05:07,345 - INFO - -> 1. Recréation des matrices H/W pour K=5 avec seed_init=42...
2025-04-11 15:05:07,518 - INFO -    Matrices d'initialisation finales H/W créées avec succès.
2025-04-11 15:05:07,519 - INFO - -> 2. Initialisation du modèle NMF final (K=5) avec random_state=1...
2025-04-11 15:05:07,519 - INFO -    Modèle NMF final initialisé.
2025-04-11 15:05:07,520 - INFO - -> 3. Entraînement (fit) du modèle NMF final...
2025-04-11 15:05:12,562 - INFO -    Modèle NMF final entraîné avec succès en 5.04s.
2025-04-11 15:05:12,565 - INFO -    Erreur de reconstruction finale : 8703.4261
2025-04-11

In [70]:
!python -m pip show gensim

Name: gensim
Version: 4.3.3
Summary: Python framework for fast Vector Space Modelling
Home-page: https://radimrehurek.com/gensim/
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.12/site-packages
Requires: numpy, scipy, smart-open
Required-by: octis


In [72]:
gensim_dictionary

##**NMF**

In [21]:
# Initialisation des variables globales utilisées pour stocker les résultats de la factorisation NMF et les données associées.

# all_nmf_H : Dictionnaire global où les matrices H (composantes thématiques) générées par le modèle NMF
# pour chaque nombre de topics sont stockées. La clé correspond au nombre de topics (e.g., 5, 10, etc.),
# et la valeur est la matrice H générée pour ce nombre de topics.
all_nmf_H = {}

# all_nmf_W : Dictionnaire global où les matrices W (représentant les documents dans l'espace des topics)
# sont stockées. Chaque clé correspond à un nombre de topics, et chaque valeur est la matrice W associée.
all_nmf_W = {}

# coherence_scores : Dictionnaire global associant à chaque nombre de topics un score de cohérence
# calculé par une métrique de type “fenêtre glissante” (par exemple c_npmi ou c_uci).
# Nous utilisons ici NMF, car il est fréquent qu'un même document soit associé à plusieurs topics.
# Dans cette situation, mesurer la qualité à l'échelle d'un document entier (doc-based) pourrait
# masquer des co-occurrences thématiques plus fines. Une approche en fenêtre glissante s'avère
# donc plus appropriée pour évaluer la cohérence locale des mots-clés liés à chaque topic.
coherence_scores = {}

# Appel de la fonction determine_nmf avec une liste de nombres de topics à tester ([10, 15]),
# en spécifiant les nouveaux paramètres alpha_W, alpha_H et l1_ratio.
# La fonction accepte également n_top_words et window_size si besoin (avec des valeurs par défaut de 15 et 100).
# Exemple d’appel pour tester respectivement 10 et 15 topics :
# determine_nmf([5, 7, 10, 12, 15, 20], alpha_W=0.3, alpha_H=0.3, l1_ratio=0.0, n_top_words=15, window_size=100)
nmf_models = determine_nmf([7, 12, 15], alpha_W=0.0, alpha_H=0.0, l1_ratio=0.0)

PROCESSUS DES TOPICS:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
for nb_topics in nmf_models:
    print(nmf_models[nb_topics].reconstruction_err_)

10349.582406870564
10297.523126394293
10271.301325119022


In [23]:
print(coherence_scores) # IL FAUT LES METTRE SUR LE DISQUE

{7: 0.12059232985642639, 12: 0.11454644725085544, 15: 0.1106162920574208}


In [24]:
#aller chopper automatiquement le max c_npmi ?

In [25]:
# These texts specifically originate from American right-wing YouTube content.

In [26]:
preprompt = "Ces textes proviennent de comptes politiques actifs sur les réseaux sociaux français pendant la campagne de l'élection présidentielle de 2022."

In [27]:
# Ce script est conçu pour aider les analystes à extraire des phrases représentatives
# après l'entraînement d'un modèle NMF. L’objectif global est de sélectionner,
# pour chaque thème (ou topic) identifié par le NMF, un ensemble de phrases
# pertinentes et peu redondantes. Pour cela, le code met à jour des listes de
# termes unigrams, génère une matrice TF-IDF de l’ensemble des phrases, puis
# utilise les scores NMF pour repérer les phrases les plus discriminantes de chaque
# topic, en filtrant celles qui sont trop similaires entre elles. Les sorties,
# notamment les phrases et leurs scores de pertinence, permettent ensuite
# de réaliser des synthèses qualitatives sur les topics détectés.
topic_labels_by_config = extract_relevant_sentences_and_titles(nmf_models)

In [28]:
# Ce code enregistre sur le disque l'esnsemble des matrices H (topics-termes) et
# des matrices W (documents-topics) sur le disque. Il y a donc une matrice H et
# une matrice W par configuration (nombre de topics). Ce sont les seuls objets
# dont nous aurons besoin pour les analyses, en plus des données du corpus.
with open(results_path + base_name + '_RAW/all_nmf_W.pkl', 'wb') as f:
    pickle.dump(all_nmf_W, f)

with open(results_path + base_name + '_RAW/all_nmf_H.pkl', 'wb') as f:
    pickle.dump(all_nmf_H, f)

#**PHASE D'ANALYSES**

## **RÉCUPÉRATION DES MATRICES H ET W ENREGISTRÉES SUR LE DISQUE**

In [29]:
# Récupération des matrices W et H depuis le disque.
with open(results_path + base_name + '_RAW/all_nmf_W.pkl', 'rb') as f:
    all_nmf_W = pickle.load(f)

with open(results_path + base_name + '_RAW/all_nmf_H.pkl', 'rb') as f:
    all_nmf_H = pickle.load(f)

##**EXTRACTION DES PHRASES CARACTÉRISTIQUES**



In [30]:
write_documents_infos()

ÉCRITURE DES FICHIERS SUR LE DISQUE:   0%|          | 0/3 [00:00<?, ?it/s]

##**DYNAMIQUE DES TOPICS**

In [31]:
# Le paramètre `sigma` détermine le niveau de lissage de la distribution des scores des topics.
# Lorsque `sigma='auto'`, la fonction tente de déterminer automatiquement un lissage "optimal"
# en se basant sur la moyenne des écarts-types des valeurs dans chaque période temporelle.
# Cependant, cette approche n'est pas toujours idéale, et il peut être nécessaire d'expérimenter
# avec différentes valeurs de `sigma`. Si `sigma=1`, aucun lissage n'est appliqué, laissant
# les valeurs brutes du DataFrame intactes.
#
# Il est important de noter qu'il n'existe pas de lissage "optimal" universel. Le lissage permet
# de rendre les dynamiques des topics plus intelligibles visuellement, en supprimant les variations
# erratiques et en montrant les tendances générales des topics au fil du temps. Par exemple,
# cela peut aider à mettre en évidence un topic dominant au début ou à la fin de la période analysée.
#
# Cependant, un lissage trop fort peut masquer des variations significatives au sein des données
# et donner une impression erronée de stabilité. Une fois lissée, il devient plus difficile
# de distinguer si une zone de forte intensité est due à une présence soutenue du topic tout au
# long de la période ou à un événement ponctuel particulièrement intense.
for vertical_normalization in [False, True]:
    for horizontal_normalization in [False, True]:
        create_chrono_topics(sigma='auto', 
                             apply_vertical_normalization=vertical_normalization,
                             apply_horizontal_normalization=horizontal_normalization)

CONFIGURATIONS PROCESSÉES:   0%|          | 0/3 [00:00<?, ?it/s]

CONFIGURATIONS PROCESSÉES:   0%|          | 0/3 [00:00<?, ?it/s]

CONFIGURATIONS PROCESSÉES:   0%|          | 0/3 [00:00<?, ?it/s]

CONFIGURATIONS PROCESSÉES:   0%|          | 0/3 [00:00<?, ?it/s]

##**DYNAMIQUE DE GROUPES**



In [32]:
# Le paramètre `sigma` détermine le niveau de lissage de la distribution des scores des journaux ou autres entités au fil du temps.
# Lorsque `sigma='auto'`, la fonction tente de déterminer automatiquement un lissage "optimal" basé sur la moyenne
# des écarts-types des valeurs dans chaque période. Cependant, cette méthode n'est pas toujours efficace, et il
# peut être utile d'expérimenter avec différentes valeurs de `sigma`.
#
# Par exemple, si `sigma=1`, aucun lissage n'est appliqué et les valeurs brutes sont utilisées, laissant ainsi
# apparaître toutes les fluctuations, même mineures. Si `sigma=30`, cela signifie que l'on applique un fort lissage
# qui va aplanir considérablement les données, en supprimant une bonne partie des variations quotidiennes et
# en montrant plutôt des tendances à plus long terme. Plus `sigma` est grand, plus la courbe est "étalée" et
# lissée, ce qui permet de voir des tendances générales, mais au détriment de la détection des pics ponctuels.
#
# Il est important de souligner qu'il n'existe pas de lissage "optimal" universel. Le lissage permet de rendre
# les dynamiques de la publication des journaux plus intelligibles en supprimant les fluctuations erratiques
# et en exposant les tendances générales des journaux au fil du temps. Par exemple, cela permet de visualiser
# les périodes où certains journaux dominent la couverture médiatique.
#
# Cependant, un lissage trop important peut atténuer des variations cruciales dans les données et nuire à
# l'analyse des événements ponctuels ou des périodes d'intensité médiatique accrue. Une fois lissée,
# il devient difficile de savoir si une zone de forte intensité est le résultat d'une couverture continue
# ou d'un événement médiatique exceptionnellement intense.
#
# Le paramètre `group_column` est pertinent uniquement lorsque `source_type='csv'`. Dans ce cas, il s'agit du
# nom (string) d'une colonne catégorielle sur laquelle regrouper les données, similaire à la colonne "journal"
# dans le cas de Istex ou de Europresse. Par exemple, si votre CSV contient une colonne "publisher" qui répertorie le nom de
# l'éditeur ou de la source du document, vous pouvez utiliser `group_column='publisher'` pour agréger
# temporellement les données par éditeur. De même, si vous avez une colonne "theme" pour catégoriser
# les articles par sujet, `group_column='theme'` permettra de visualiser l'évolution temporelle
# par sujet. Si `group_column` n'est pas fourni ou n'existe pas dans `columns_dict`, un message
# d'erreur est affiché et la fonction s'interrompt. Ce mécanisme permet de créer une agrégation temporelle
# par catégorie, facilitant l'analyse par groupes définis (par exemple, par titre de publication, thème, pays, etc.).
for vertical_normalization in [False, True]:
    for horizontal_normalization in [False, True]:
        create_chrono_group_column(group_column='political_party', 
                                   sigma='auto', 
                                   apply_vertical_normalization=vertical_normalization,
                                   apply_horizontal_normalization=horizontal_normalization)

## **SPATIALISATION DES TOPICS PAR PCA**

In [33]:
# Cette fonction vise à analyser et visualiser les relations entre topics en appliquant une PCA sur les matrices
# produites par la factorisation NMF (matrices W et H), chaque projection ayant un objectif distinct :
#
# 1) **PCA sur W** :
#    - La matrice W décrit comment les topics sont distribués à travers l'ensemble des documents.
#    - En appliquant une PCA sur W, nous cherchons à rapprocher les topics qui partagent une distribution
#      similaire sur les documents, c'est-à-dire ceux qui apparaissent de manière conjointe dans les mêmes
#      ensembles documentaires. Cela fournit une vision "macro" des relations entre topics dans l'espace documentaire.
#
# 2) **PCA sur H** :
#    - La matrice H décrit comment les mots contribuent à la définition de chaque topic.
#    - En appliquant une PCA sur H, nous cherchons à rapprocher les topics ayant un vocabulaire similaire,
#      c'est-à-dire ceux qui partagent des mots-clés ou des caractéristiques linguistiques proches.
#      Cette analyse met en évidence des proximités sémantiques entre les topics.
#
# Ces deux visualisations permettent d'explorer les relations entre topics sous deux angles complémentaires :
# - La PCA sur W éclaire les co-apparitions thématiques dans les documents.
# - La PCA sur H révèle les similitudes lexicales ou sémantiques entre topics.
#
# En somme, cette étape enrichit l'interprétation des topics NMF en offrant une vue synthétique de leurs
# relations documentaires et lexicales.
plot_pca('W')
plot_pca('H')

Impossible d'obtenir un renderer valide initialement (FontProperties.__init__() got an unexpected keyword argument '_internal.classic_mode'), forçage d'un dessin...
[Passe 1] Application d'une limite de temps de 600000 ms.
[Passe 1] Lancement de la résolution...
[Passe 1] Résolution terminée avec le statut : 0
[Passe 1] Distance minimale trouvée (approximative) : 0.0
[Passe 2] Application d'une limite de temps de 600000 ms.
[Passe 2] Lancement de la résolution...
[Passe 2] Résolution terminée avec le statut : 0
[2passes] Solution finale extraite avec 7 labels placés.
Impossible d'obtenir un renderer valide initialement (FontProperties.__init__() got an unexpected keyword argument '_internal.classic_mode'), forçage d'un dessin...
[Passe 1] Application d'une limite de temps de 600000 ms.
[Passe 1] Lancement de la résolution...
[Passe 1] Résolution terminée avec le statut : 0
[Passe 1] Distance minimale trouvée (approximative) : 0.08376
[Passe 2] Application d'une limite de temps de 60000

## **ANALYSE DE SENTIMENTS**

In [34]:
# Cette fonction effectue une analyse de sentiments sur une liste de documents textuels, en s’appuyant
# sur un modèle BERT multilingue (nlptown/bert-base-multilingual-uncased-sentiment). Elle parcourt chaque texte,
# le tokenize et le passe au pipeline d’analyse afin d’obtenir une note d’étoiles (de 1 à 5) reflétant
# le sentiment.
#
# Attention : par défaut, le modèle BERT utilisé est limité à 512 tokens. Si le texte dépasse cette limite,
# il sera automatiquement tronqué aux 512 premiers tokens par le tokenizer et le pipeline. Par conséquent,
# seuls ces 512 premiers tokens seront pris en compte dans le calcul du sentiment, ce qui peut potentiellement
# biaiser l’analyse pour les textes très longs. Si un traitement plus complet est nécessaire, il conviendra
# de segmenter le texte en plusieurs morceaux de taille inférieure ou égale à 512 tokens et d’analyser
# chaque segment individuellement.
#
# Après l’analyse, la fonction calcule une moyenne des scores des documents, transforme les dates,
# puis génère des graphiques de sentiment dans le temps et sous forme de cartes thermiques (heatmaps)
# en fonction de différents paramètres.
process_sentiments()

Device set to use mps


device mps


Processing Documents:   0%|          | 0/86338 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

##**FORÊTS ALÉATOIRES AVEC ANALYSE DES RÉSIDUS NORMALISÉS**

In [35]:
create_box_plots(group_column='political_party')

Processing topics:   0%|          | 0/7 [00:00<?, ?it/s]

Processing topics:   0%|          | 0/12 [00:00<?, ?it/s]

Processing topics:   0%|          | 0/15 [00:00<?, ?it/s]

In [36]:
random_forests_residuals_analysis(group_column='political_party')

RÉGRESSIONS : ANALYSE DES RÉSIDUS:   0%|          | 0/7 [00:00<?, ?it/s]

RÉGRESSIONS : ANALYSE DES RÉSIDUS:   0%|          | 0/12 [00:00<?, ?it/s]

RÉGRESSIONS : ANALYSE DES RÉSIDUS:   0%|          | 0/15 [00:00<?, ?it/s]